# 4. Escalas, eixos e legendas

Codificação visual &ndash; mapear dados para variáveis visuais como posição, tamanho, formato, ou cor &ndash; é o coração pulsante da visualização de dados. O motor que realmente performa o mapeamento é a escala (`scale`): a função que recebe os dados como entrada (o *domínio* da escala) e retorna um valor visual, como a posição de um pixel ou cor RGB, como saída (a *imagem* da escala). Evidentemente, a visualização é inútil se ninguém consegue entender o que ela quer transmitir! Complementando as marcas gráficas, um gráfico precisa de elementos de referência, ou *guias*, que permitem leitores lerem o gráfico. Guias como os *eixos* (que exibem escalas como intervalos no espaço) e *legendas* (que exibem escalas com uma faixa de cor, tamanho, ou formato), são heróis anônimos de visualização efetiva de dados!

Neste capítulo, nós vamos explorar as opções que o Altair oferece para permitir designs custumizados de mapeamento de escalas, eixos e legendas, usando e rodando um exemplo sobre a efetividade de antibióticos.

_Esse notebook é parte do [Currículo de Visualização de Dados](https://github.com/uwdata/visualization-curriculum)._

In [ ]:
import pandas as pd
import altair as alt

## 4.1. Dados dos Antibióticos

Depois da Segunda Guerra Mundial, antibióticos foram considerados "drogas milagrosas", por serem um remédio fácil para doenças consideradas intratáveis. Para aprender qual droga funcionava mais efetivamente contra cada infecção bacteriana, a performance dos 3 antibióticos mais populares contra 16 das bactérias causadoras de infecções mais comuns foi coletada.

Nós vamos usar um conjunto de dados de antibióticos da [coleção do vega-datasets](https://github.com/vega/vega-datasets). No exemplo abaixo, nós vamos passar o URL diretamente para o Altair.

In [ ]:
antibioticos = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/burtin.json'

Nós podemos primeiro carregar os dados com Pandas para ver o conjunto de dados inteiro e nos familiarizarmos com os campos de dados disponíveis:

In [ ]:
pd.read_json(antibioticos)

,Bacteria,Penicillin,Streptomycin,Neomycin,Gram_Staining,Genus
0,Aerobacter aerogenes,870.000,1.00,1.600,negative,other
1,Bacillus anthracis,0.001,0.01,0.007,positive,other
2,Brucella abortus,1.000,2.00,0.020,negative,other
3,Diplococcus pneumoniae,0.005,11.00,10.000,positive,other
4,Escherichia coli,100.000,0.40,0.100,negative,other
5,Klebsiella pneumoniae,850.000,1.20,1.000,negative,other
6,Mycobacterium tuberculosis,800.000,5.00,2.000,negative,other
7,Proteus vulgaris,3.000,0.10,0.100,negative,other
8,Pseudomonas aeruginosa,850.000,2.00,0.400,negative,other
9,Salmonella (Eberthella) typhosa,1.000,0.40,0.008,negative,Salmonella


O valor numérico indicado na tabela é a [Concentração Mínima Inibitória (CMI)](https://pt.wikipedia.org/wiki/Concentra%C3%A7%C3%A3o_inibit%C3%B3ria_m%C3%ADnima), uma medida de eficácia do antibiótico, que representa a concentração de antibiótico (em microgramas por mililitro) necessária para previnir o crescimento da bactéria em laboratório. A reação da bactéria para um procedimento chamado [Teste de Gram](https://pt.wikipedia.org/wiki/T%C3%A9cnica_de_Gram) é descrita pela coluna `Gram_Staining`. Bactérias que ficam azul ou violeta no teste respondem positivamente (`positive`), enquanto as que não reagem assim são negativas (`negative`).

Enquanto examinamos diferentes visualizações desse conjunto de dados, se pergunte: O que podemos aprender sobre a eficácia relativa dos antibióticos? O que podemos aprender sobre as espécies bacterianas baseado na sua resposta ao antibiótico?

# 4.2 Ajustando escalas e eixos
## 4.2.1 Plotando a resistência a antibióticos: ajustando o tipo de escala

Vamos começar observando um gráfico de pontos simples da CMI para Neomicina (Neomycin).

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q')
)

alt.Chart(...)

*Podemos ver que os valores da CMI abrangem ordens de magnitude diferentes: a maior parte dos pontos está agrupada à esquerda, com alguns valores discrepantes maiores à direita.*

Por padrão, o Altair faz mapeamento linear entre os valores do domínio e a distância em pixels. Nesse caso, podemos ver que a distância entre 0 e 10 no eixo X em pixels é exatamente igual à distância entre 30 e 40. Enquanto isso é um padrão bom e o mais utilizado, no nosso gráfico, perdemos precisão ao observar os nossos dados, já que nosso eixo X é tão grande que o intervalo onde encontramos a maior parte dos nossos pontos fica achatado.

Para visualizar, então, nossos dados de maneira melhor, podemos aplicar uma transformação de escala no nosso eixo.


Para aplicar uma transformação de escala, ao criar o eixo, definimos seu atributo de escala (`scale`), usando o método `alt.Scale`, e definindo seu tipo (`type`).


Este seria o resultado se usássemos uma escala do tipo raiz quadrada (`type = 'sqrt'`). Distâncias em pixel agora correspondem à raiz quadrada da distância no domínio:

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale = alt.Scale(type='sqrt'))
)

alt.Chart(...)

*Os pontos à esquerda estão melhor distribuídos agora, mas ainda temos discrepâncias bem grandes no gráfico.*

Vamos tentar usar uma escala [logarítmica](https://pt.wikipedia.org/wiki/Escala_logar%C3%ADtmica) (`type = 'log'`) agora:

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

*Agora, nossos dados estão bem distribuídos ao longo do gráfico, e ainda conseguimos ver a diferença entre a CMI da Neomicina para diferentes bactérias.*

Numa escala linear, se uma distância de 5 unidades no seu eixo equivale a uma distância de 10 pixels, uma distância de 10 unidades vai equivaler a uma distância de 20 pixels, proporcionalmente. A escala logarítmica trabalha de maneira diferente, se baseando no mapeamento entre adição e multiplicação usando a propriedade matemática $log(a) + log(b) = log(ab)$. Como resultado disso, temos que se a distância visual entre dois pontos é de K pixels no seu gráfico, a distância numérica, nos seus dados, é na verdade uma proporção de K, onde K é a base escolhida para o nosso logaritmo (por padrão é 10, mas pode ser mudada definindo uma `base` na hora de usar o método `alt.Scale` com `type = 'log'`).

Visualize assim: temos 3 pontos A, B, C, que valem respectivamente 10, 100 e 1000 unidades. Em um eixo de um gráfico, na escala linear, se a distância em pixels entre A e B é de 10 pixels, a distância em pixels entre B e C é de 100 pixels, pois $C-B=10(B-A)$. Na escala logarítmica de base 10, no entanto, se a distância entre A e B é de 10 pixels, a distância entre B e C é de 10 pixels também, pois $\frac{C}{B} = \frac{B}{A}$.

## 4.2.2 Decorando um eixo

Para antibióticos, quanto menos usamos para matar uma bactéria, melhor esse antibiótico é. Considerando que gostamos normalmente de ver o melhor mais acima/à direita em um gráfico, para nos adequar a esse fator, teríamos que inverter o eixo X do nosso gráfico, de forma que as menores CMIs fiquem a esquerda e as maiores à direita.


Fazemos isso definindo seu atributo de ordenação (`sort`) como decrescente (`descending`):

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort = 'descending',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

*Nosso gráfico está começando a ficar confuso: estamos usando um eixo com escala logarítmica, em ordem inversa e sem indicação de unidades ainda!*

Vamos começar então a dar mais informações ao nosso gráfico adicionando um atributo de título (`title`) ao nosso eixo:

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort = 'descending',
          title = 'CMI da Neomicina (μg/ml, escala logarítmica reversa)',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

Bem melhor.

Podemos também mudar de lugar o nosso eixo. Por padrão, o Altair coloca o eixo X e suas informações na parte inferior do gráfico, mas podemos alterar isso adicionando um atributo de eixo (`axis`) no nosso código, usando o método `alt.Axis` e definindo sua orientação (`orient`) como superior (`'top'`):

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort = 'descending',
          axis = alt.Axis(orient = 'top'),
          title = 'CMI da Neomicina (μg/ml, escala logarítmica reversa)',
          scale = alt.Scale(type='log'))
)

alt.Chart(...)

O mesmo se aplica ao eixo Y, que normalmente está posicionado à esquerda mas pode ser posicionado à direita com a orientação `'right'`.

## 4.2.3 Comparação de antibióticos: Ajustar Grades, contagens de Tick e dimensionamento


_Como é que a neomicina se compara a outros antibióticos, como a estreptomicina e a penicilina?_

Para começar a responder a essas questões, podemos criar um gráfico de dispersão, adicionando um eixo vertical que codifica outro antibiótico e espelha nosso eixo x para a neomicina.

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Streptomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Estreptomicina (μg/ml, escala logarítmica reversa)')
)

alt.Chart(...)

*Podemos ver que ambas a Neomicina e a Estreptomicina aparentam ser bem correlacionadas, já que as cepas bacteriais estudadas respondem de maneira similar a ambos antibióticos.*


Prosseguindo, vamos comparar a Neomicina e a Penicilina.

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log'),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
)

alt.Chart(...)

*Vemos agora uma resposta diferente: algumas bactérias respondem bem a um antibiótico enquanto outras não.*

Apesar do gráfico ser útil, podemos melhorar. Os eixos x e y usam a mesma unidade, mas tem dimensões diferentes (a largura do gráfico é maior que a altura) e domínios diferentes (0.001 até 100 no eixo x, e 0.001 até 1000 para o eixo y).

Vamos então igualar os eixos: podemos explicitar diretamente valores para altura (`height`) e largura (`width`) em pixels nas propriedades (`properties`) do gráfico e especificar o domínio (`domain`) na escala.


In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
).properties(width=250, height=250)

alt.Chart(...)

_O gráfico resultante é mais equilibrado e menos propenso a erros de interpretação._

No entanto, as linhas de grade estão densas demais agora. Se quisermos remover completamente as linhas da grade (`grid`) do gráfico, podemos adicionar `grid=False` ao atributo do eixo. Mas e se, em vez disso, quisermos reduzir o número de marcas de seleção, por exemplo, incluindo apenas linhas para cada ordem de grandeza?

Para alterar o número de marcas, podemos especificar o número de marcações (`tickCount`) ao chamar o método `alt.Axis` no nosso código. A propriedade `tickCount` é tratada como uma sugestão para o Altair, a ser considerada juntamente com outros aspectos, tais como a utilização de intervalos agradáveis e de fácil utilização. Podemos não obter exatamente o número de marcas de escala que solicitamos, mas devemos obter algo próximo.

In [ ]:
alt.Chart(antibioticos).mark_circle().encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis = alt.Axis(tickCount = 5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis = alt.Axis(tickCount = 5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
).properties(width=250, height=250)

alt.Chart(...)

Ao definir o `tickCount` para 5, obtemos o efeito desejado.

Os pontos do nosso gráfico de dispersão parecem um pouco pequenos. Vamos alterar o tamanho predefinido, definindo o tamanho (`size`) da marca circular ao criar nosso gráfico. Este valor de tamanho é a área da marca em pixels.

In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa )'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)')
).properties(width=250, height=250)

alt.Chart(...)

_Aqui, definimos a área da marca circular para 80 píxeis. Ajuste o valor como achar melhor._

# 4.3 Configurando as Legendas de Cor

## 4.3.1 Cor por Teste de Gram

Vimos acima que a Neomicina é mais eficaz para algumas bactérias enquanto a Penicilina é mais eficaz para outras. Mas, como podemos determinar qual antibiótico usar se não sabemos exatamente a espécie de bactéria com a qual estamos lidando? O teste de Gram serve como um método de separar classes de bactérias!

Vamos codificar o campo dos testes de Gram, `Gram_Staining`, no canal de cor (`color`) como um tipo de dado nominal:


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N')
).properties(width=250, height=250)

alt.Chart(...)

Podemos ver que as bactérias que testam positivamente (`positive`) no teste de Gram parecem ser mais suscetíveis à Penicilina, enquanto a Neomicina é mais eficaz para bactérias que testam negativamente  (`negative`)!

O esquema de cores acima foi escolhido automaticamente para fornecer cores perceptivelmente distintas para comparações nominais (igual ou diferente). No entanto, podemos querer personalizar as cores utilizadas. Nesse caso, o teste de Gram resulta em colorações físicas distintas: as bactérias que testam negativo ficam rosa, e as que testam postivo ficam roxo.

Podemos determinar o uso dessas cores especificamente dentro da escala de `alt.Color`, especificando o domínio (`domain`) dos dados utilizados e o alcance (`range`) das cores selecionadas pelo Altair:


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple'])
    )
).properties(width=250, height=250)

Por padrão, as legendas são posicionadas no lado direito do gráfico. Similar aos eixos, podemos alterar a orientação (`orient`) da legenda dentro do método `alt.Legend`:


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=alt.Legend(orient='left')
    )
).properties(width=250, height=250)

alt.Chart(...)

Também podemos remover completamente uma legenda especificando `legend=None`:



In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gram_Staining:N',
          scale=alt.Scale(domain=['negative', 'positive'], range=['hotpink', 'purple']),
          legend=None
    )
).properties(width=250, height=250)

alt.Chart(...)

## 4.3.2 Cor por Espécie

Até agora, consideramos a eficácia dos antibióticos. Vamos mudar a abordagem e fazer uma pergunta diferente: o que a resposta aos antibióticos pode nos ensinar sobre as diferentes espécies de bactérias?

Para começar, vamos codificar `Bacteria` (um campo de dados nominal) usando o canal de `color`:


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:N')
).properties(width=250, height=250)

alt.Chart(...)

O resultado está um pouco confuso! Existem bactérias suficientes para que o Altair comece a repetir cores de sua paleta padrão de 10 cores para valores nominais.

Para usar cores personalizadas, podemos atualizar a propriedade `scale` da codificação de cor. Uma opção é fornecer valores explícitos de `domain` e `range` da escala para indicar mapeamentos de cores precisos por valor, como fizemos acima para `Gram Staining`. Outra opção é usar um esquema de cores alternativo. O Altair inclui uma variedade de esquemas de cores integrados. Para uma lista completa, consulte a documentação do esquema de cores do [Vega](https://vega.github.io/vega/docs/schemes/#reference).

Vamos tentar alternar para um esquema integrado de 20 cores, `tableau20`, e definir isso usando a propriedade de escala de esquema (`scheme`).


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Nemicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

Agora temos uma cor única para cada bactéria, mas o gráfico ainda está confuso. Entre outros problemas, a codificação não leva em conta bactérias que pertencem ao mesmo gênero. No gráfico acima, as duas cepas diferentes de Salmonella possuem matizes muito diferentes (verde-azulado e rosa), apesar de serem biologicamente próximas.

Para testar um `scheme` diferente, também podemos alterar o tipo de dado de nominal para ordinal. O `scheme` ordinal padrão usa tons de azul, variando do claro ao escuro:


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:O')
).properties(width=250, height=250)

alt.Chart(...)

Alguns desses tons de azul podem ser difíceis de distinguir.

Para obter cores mais diferenciadas, podemos experimentar alternativas ao esquema padrão `blues`. O esquema `viridis` varia tanto em matiz quanto em luminância:


In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Bacteria:O',
          scale=alt.Scale(scheme='viridis'))
).properties(width=250, height=250)

alt.Chart(...)

Bactérias do mesmo gênero agora possuem cores mais semelhantes do que antes, mas o gráfico ainda continua confuso. Há muitas cores, é difícil identificá-las com precisão na legenda, e duas bactérias podem ter cores semelhantes, mas pertencer a gêneros diferentes.


## 4.3.3. Colorindo por gênero

Vamos tentar colorir por gênero ao invés de colorir por bactéria. Para isso, adicionaremos o transformador `calculate` na criação do gráfico, separando o nome da bactéria por caracteres de espaço e selecionando o primeiro elemento da lista, assim tendo o gênero da bactéria. Podemos então codificar os gêneros resultantes, presentes no campo `Gênero`, utilizando o esquema de cores `tableau20`.

(Note que a base de dados "antibiotics" tem um campo de gênero, mas ignoraremos ele para poder fazer uma melhor exploração nas transformações de dados do Altair.)

In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).transform_calculate(
    Gênero='split(datum.Bacteria, " ")[0]'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gênero:N',
          scale=alt.Scale(scheme='tableau20'))
).properties(width=250, height=250)

alt.Chart(...)

_Hmmm... Por mais que os dados estejam melhor agregados pelos gêneros, essa cacofonia de cores não parece muito útil._

_Se repararmos cuidadosamente em alguns do gráficos anteriores poderemos ver que apenas um grupo seleto de bactérias compartilham gênero com alguma outra: Salmonella, Staphylococcus, e Streptococcus. Para trazer um foco à nossa comparação adicionaremos cores apenas para esses gêneros repetidos._

Adicionemos outro transformador `calculate` que mantém o gênero se for um dos valores repetidos e caso contrário transforma em `"Outro"`.

Além disso, podemos adicionar codificadores de cores personalizados usando intervalos específicos com `domain` e `range` para definir as cores.

In [ ]:
alt.Chart(antibioticos).mark_circle(size=80).transform_calculate(
  Split='split(datum.Bacteria, " ")[0]'
).transform_calculate(
  Gênero='indexof(["Salmonella", "Staphylococcus", "Streptococcus"], datum.Split) >= 0 ? datum.Split : "Outro"'
).encode(
    alt.X('Neomycin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Neomicina (μg/ml, escala logarítmica reversa)'),
    alt.Y('Penicillin:Q',
          sort='descending',
          scale=alt.Scale(type='log', domain=[0.001, 1000]),
          axis=alt.Axis(tickCount=5),
          title='CMI da Penicilina (μg/ml, escala logarítmica reversa)'),
    alt.Color('Gênero:N',
          scale=alt.Scale(
            domain=['Salmonella', 'Staphylococcus', 'Streptococcus', 'Outro'],
            range=['rgb(76,120,168)', 'rgb(84,162,75)', 'rgb(228,87,86)', 'rgb(121,112,110)']
          ))
).properties(width=250, height=250)

alt.Chart(...)

_Nós agora temos um gráfico muito mais revelador que só foi possivel graças às personalizações feitas nos eixos e legendas. Tire um momento para analisar o gráfico acima. Consegue perceber algum agrupamento surpreendente?_

_A parte superior esquerda apresenta um agrupamento de bactérias do gênero Streptococcus representados pela cor vermelha com uma bactéria do grupo "Outros" junta. Enquanto isso, na região do meio mais à direita observa-se outra Streptococcus posicionada longe de seus "primos". Devemos esperar que bactérias do mesmo gênero (e presumivelmente mais parecidas geneticamente) estejam mais próximas entre elas?_

Como ocorre eventualmente, a base de dados em questão contém erros. A base de dados se baseia nas designações de espécie utilizada no início dos anos 1950. No entanto, o consenso científico mudou desde então. O ponto cinza no canto superior esquerdo? Agora é considerado Streptococcus! Aquele ponto vermleho no meio à direita afastado dos demais? Não é mais considerado Streptococcus!

É claro que, por si só, essa base de dados não justifica essas reclassificações. Entretanto, os dados contém dicas valiosas para a biologia que foram ignoradas por décadas! Visualização de dados, quando utilizada por alguém habilidoso e curioso, pode ser uma arma poderosa para descobertas..

Esse exemplo reforça algo importante: **sempre tenha um pé atrás com seus dados!**

## 4.3.4. Colorindo por resposta a antibióticos

Nós também podemos utilizar o canal das cores `color` para codificar valores quantitativos. Mas é ipmortante ter em mente que o canal das cores não costuma ser tão eficiente para valores quantitativos quanto codificadores como posição e tamanho!

Aqui está um mapa de calor básico dos valores de CMI da Penicilina para cada bactéria. Usaremos uma marcação de retângulos (`rect`) e ordenar as bactérias por valores decrescente de CMI (de mais resistente a menos resistente).

In [ ]:
alt.Chart(antibioticos).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending')
    ),
    alt.Color('Penicillin:Q')
)

alt.Chart(...)

Nós podemos ainda melhorar esse gráfico combinando métodos vistos até aqui: uma escala logaritimica, uma mudança de orientação nos eixos, um esquema de cores personalizado (`plasma`), ajuste na contagem de marcações nos eixos e texto do titulo customizado. Nós também iremos usar opções de configurações para ajustar o posicionamento do título dos eixos e o alinhamento do título da legenda.

In [ ]:
alt.Chart(antibioticos).mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(
        orient='right',     # Orienta o eixo à direita
        titleX=7,           # Define a posição X do título como 7 pixels à direita no gráfico
        titleY=-2,          # Define a posição Y do título como 2 pixels abaixo no gráfico
        titleAlign='left',  # Alinha o texto à esquerda
        titleAngle=0        # Desfaz a rotação padrão do título do eixo Y
      )
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='CMI da Penicilina (μg/ml)'
    )
)

alt.Chart(...)

Alternativamente, podemos remover o título do eixo e definir diretamente o `title` para todo o gráfico.

In [ ]:
alt.Chart(antibioticos, title='Resistência a Penicilina de Cepas Bacterianas').mark_rect().encode(
    alt.Y('Bacteria:N',
      sort=alt.EncodingSortField(field='Penicillin', op='max', order='descending'),
      axis=alt.Axis(orient='right', title=None)
    ),
    alt.Color('Penicillin:Q',
      scale=alt.Scale(type='log', scheme='plasma', nice=True),
      legend=alt.Legend(titleOrient='right', tickCount=5),
      title='CMI da Penicilina (μg/ml)'
    )
).configure_title(
  anchor='start', # anchor and left-align title
  offset=5        # set title offset from chart
)

alt.Chart(...)

# 4.4. Sumário

Resumindo tudo que foi aprendido nos notebooks até aqui sobre codificação, transformação de dados e personalização, você agora deve estar preparado para fazer uma grande variedade de gráficos estátisticos. Agora você pode inserir o Altair em atividades do dia a dia para explorar dados e passar as mensagens escondidas neles.

Interessado em aprender mais sobre esse tópico?
- Comece com a [documentação do Altair sobre visualizações customizadas](https://altair-viz.github.io/user_guide/customization.html).
- Para uma discussão complementar sobre escalas de mapeamento veja ["Introducing d3-scale"](https://medium.com/@mbostock/introducing-d3-scale-61980c51545f).
- Para uma exploração mais profunda de como os eixos e as legendas podem ser personalizados pela biblioteca Vega (que é a base para Altair e Vega-Lite), veja ["A Guide to Guides: Axes & Legends in Vega"](https://beta.observablehq.com/@jheer/a-guide-to-guides-axes-legends-in-vega)
- Para uma história fascinante sobre a base de dados sobre antibióticos trabalhada esse notebok, veja ["That's Funny...", de Wainer e Lysen](https://www.americanscientist.org/article/thats-funny) na revista _American Scientist_.

# Gráficos Individuais

## Carlos

Primeiramente vamos importar as bibliotecas necessárias

In [ ]:
import altair as alt
import pandas as pd


Para nossos estudos, utilizaremos uma base de dados obtida no site [base dos dados](https://basedosdados.org/). Foi escolhido uma base que contempla dados do saneamanto básico de Pernambuco.

In [ ]:
file_id = "12j_IuZAzITdIZPGvamLQbYaNo-YvwqKU"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

alt.data_transformers.disable_max_rows()

df = pd.read_csv(url)
df.head()


Vamos observar, primeiramente, como se da a distrubuição de água.

In [ ]:
alt.Chart(df).mark_boxplot().encode(
    y=alt.Y('populacao_urbana_atendida_agua:Q', title='População Urbana Atendida com Água')
).properties(
    title='Boxplot da População Urbana Atendida com Água'
)

Perceba que esse gráfico não diz muita coisa, na verdade ele não nos representa nada, visto que a distrubuição de água por cidade e o número de residentes não é levando em conta.

Vamos observar a distribuição de água de pernambuco no geral no decorrer dos anos.

In [ ]:
numero_agua_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),

).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_agua_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana_atendida_agua):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_agua_line + numero_agua_pontos

Podemos observar aparente crescimento no número de pessoas que receberam água durante. Porém isso não confirma nada, visto que a população também cresce ao longo do tempo. Portanto, para uma observação honesta, vamos plotar junto o crescimento da população urbana a cada ano. Além do mais, perceba que não temos dados do ano de 2000 para trás. Então vamos retirar esses dados


In [ ]:
df = df[df['ano'] > 2000]

numero_agua_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),

).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_agua_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana_atendida_agua):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)


numero_pessoas_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_pessoas_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)

numero_agua_line + numero_agua_pontos + numero_pessoas_pontos + numero_pessoas_line

Aqui percebemos eventos estranhos no dado, como a diminuição repentina de pessoas com água em casa sem ter diminuição na população no ano de 2010. É claro, essa diminuição repentina pode ser simplesmente um erro no nosso dataframe.

Vamos observar o caso dos esgotos.

In [ ]:

numero_esgoto_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_esgoto):Q', title='Número de Pessoas Atendidas com Água'),

).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_esgoto_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_esgoto):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana_atendida_esgoto):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)


numero_pessoas_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_pessoas_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana):Q')
).properties(
    title='Número de Pessoas Atendidas com Esgoto por Ano'
)

numero_esgoto_line + numero_esgoto_pontos + numero_pessoas_pontos + numero_pessoas_line

Aqui observamos a situação mais crírica apresentada. O número de pessoas com esgoto (pelo menos o que a pesquisa entende como esgoto) é menor do que o número de residentes.

Porém, todos esse gráficos não nos dão uma pespectiva em percential. Vamos observar a caracteristicas percertuais de cada cidade.

In [ ]:
df["proporcao_agua"] = df["populacao_urbana_atendida_agua"] / df["populacao_urbana"]
df["proporcao_esgoto"] = df["populacao_urbana_atendida_esgoto"] / df["populacao_urbana"]


df_long = df.melt(id_vars=["ano", "id_municipio"],
                  value_vars=["proporcao_agua", "proporcao_esgoto"],
                  var_name="Serviço", value_name="Proporção")

selecao_ano = alt.param(
    name="ano_escolhido",
    bind=alt.binding_select(options=sorted(df["ano"].unique()), name="Ano: "),
    value=df["ano"].min()
)
agua = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida", scale=alt.Scale(domain=[0, 1.4])),
    color=alt.value("blue"),
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".2f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_agua") & (alt.datum.ano == selecao_ano)
)

esgoto = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida"),
    color=alt.value("green"),
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".2f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_esgoto") & (alt.datum.ano == selecao_ano)
)


grafico = (agua + esgoto).add_params(selecao_ano).properties(
    title="Cobertura de Água e Esgoto nos Municípios",
    width=1200,
    height=800
)

grafico

Observe que muitas cidades em vários anos não tem nada de esgoto. Isso parece muito estanho, muito provavelmente exitem várias colunas na base de dados onde não temos informação alguma sobre esgotos na cidade. Vamos mudar o dados e remover as linhas sem iformação de esgoto usando o método `dropna`.

In [ ]:
df = df.dropna(subset=["populacao_urbana_atendida_esgoto"])


df["proporcao_agua"] = df["populacao_urbana_atendida_agua"] / df["populacao_urbana"]
df["proporcao_esgoto"] = df["populacao_urbana_atendida_esgoto"] / df["populacao_urbana"]


df_long = df.melt(id_vars=["ano", "id_municipio"],
                  value_vars=["proporcao_agua", "proporcao_esgoto"],
                  var_name="Serviço", value_name="Proporção")

selecao_ano = alt.param(
    name="ano_escolhido",
    bind=alt.binding_select(options=sorted(df["ano"].unique()), name="Ano: "),
    value=df["ano"].min()
)


agua = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida (%)", scale=alt.Scale(domain=[0, 1.4])),
    color=alt.value("blue"),  # Define a cor fixa para água
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".2f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_agua") & (alt.datum.ano == selecao_ano)
)

esgoto = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida (%)"),
    color=alt.value("green"),  # Define a cor fixa para esgoto
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".1f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_esgoto") & (alt.datum.ano == selecao_ano)
)

# Combina as duas camadas no mesmo gráfico
grafico = (agua + esgoto).add_params(selecao_ano).properties(
    title="Cobertura de Água e Esgoto nos Municípios (%)",
    width=1200,
    height=800
)

grafico

## Daniel Couto

In [ ]:
import altair as alt
import pandas as pd

Para a minha análise utilizarei uma base do Kaggle, com os dados de usuários de um determinado aplicativo de relacionamento. A base possui colunas como: Idade, gênero, altura, objetivo da pessoa no app e etc.

In [ ]:
df_app_namoro = pd.read_csv('app_namoro.csv')
# Link para a base de dados: https://www.kaggle.com/datasets/anandshaw2001/dating-dataset?resource=download
df_app_namoro = pd.DataFrame(df_app_namoro)
df_app_namoro.head()


#### Estatísticas de resumo

Vamos agora calcular medidas de resumo de algumas das variveis do DataFrame. E para uma visão geral das variáveis quantitativas podemos usar também o `.describe()`.

In [ ]:
df_app_namoro.describe()

In [ ]:
import numpy as np
media_idade = np.mean(df_app_namoro['Age'])
desvio_idade = np.std(df_app_namoro['Age'])
print(f'Média idade = {media_idade:.1f} anos'.replace('.',','))
print(f'Desvio padrão das idades = {desvio_idade:.1f} anos'.replace('.',','))
print()

media_altura = np.mean(df_app_namoro['Height'])*0.3048 #Usando a altura em metros
altura_max = max(df_app_namoro['Height'])*0.3048
desvio_altura = np.std(df_app_namoro['Height'])*30.48 #Usando a altura em centimetros
print(f'Média altura = {media_altura:.2f} m'.replace('.',','))
print(f'Maior altura = {altura_max:.2f} m'.replace('.',','))
print(f'Desvio padrão das alturas = {desvio_altura:.1f} cm'.replace('.',','))
print()

media_altura_homem = np.mean(df_app_namoro.loc[df_app_namoro['Gender'] == "Male"]['Height'])*0.3048
media_altura_mulher = np.mean(df_app_namoro.loc[df_app_namoro['Gender'] == "Female"]['Height'])*0.3048
print(f'Média altura dos homens = {media_altura_homem:.3f} m'.replace('.',','))
print(f'Média altura das mulheres = {media_altura_mulher:.3f} m'.replace('.',','))
print()

moda_interesses = df_app_namoro['Interests'].mode()
print(f'Conjunto de interesses modal: {moda_interesses}') # Essa maneira não calcula exatamente o interesse modal, e sim o conjunto de interesses modal
print()

# Para resolver isso, abrimos as listas em linhas separadas com cada interesses
interests_exploded = df_app_namoro.explode('Interests')
moda_interesse = interests_exploded['Interests'].mode()
print(f'Interesse modal: {moda_interesse}')
print()

filho = df_app_namoro['Children'].value_counts('Yes')
print(filho)

#### Análise Unidimensional

Vamos agora analisar a variável `Looking For` que nos diz o que o usuário procura no aplicativo.'

In [ ]:
objetivo = alt.Chart(df_app_namoro).mark_bar().encode(
    alt.X('Looking For'),
    alt.Y('count()'),
    color = 'Looking For'
)
objetivo

Um histograma foi a escolha mais intuitiva para uma informação de contagem e comparação entre categorias. E as cores apesar de estarem passando a mesma informação do eixo x, ajuda na visualização e entendimento.

Podemos fazer algumas melhorias no nosso gráfico, como botar títulos, traduzir a legenda, escolher as cores, redimensionar o gráfico e ainda separar entre homens e mulheres para compará-los.

In [ ]:
objetivo = alt.Chart(df_app_namoro).transform_calculate(
    traducao="datum['Looking For'] == 'Casual Dating' ? 'Encontro Casual' : "
             "datum['Looking For'] == 'Marriage' ? 'Casamento' : "
             "datum['Looking For'] == 'Long-term Relationship' ? 'Relacionamento Sério' : "
             "'Amizade'",
    genero_traduzido="datum.Gender === 'Female' ? 'Mulher' : 'Homem'"
).mark_bar().encode(
    alt.X('traducao:N', title="Interesses"),
    alt.Y('count()', title="Quantidade de Usuários"),
    alt.Color('traducao:N',
        scale=alt.Scale(
            domain=["Encontro Casual", "Casamento", "Relacionamento Sério", "Amizade"],
            range=["#eb584d", "#e0c641","#5c58e0" ,"#55e08f" ]
        ),
        legend=alt.Legend(title="Tipo de Interesse")
    ),
    column=alt.Column("genero_traduzido:N", header=alt.Header(title="Gênero"))
).properties(
    height = 450,
    width=225,
    title= alt.TitleParams(
        text="Procura dos usuários de app de relacionamento",
        anchor="middle")
)

objetivo


#### Análise Bidimensional

Nessa análise vamos comparar o nível de escolaridade com a intenção de ter filhos.

In [ ]:
escolaridade_filhos = alt.Chart(df_app_namoro).mark_rect().encode(
    alt.X('Education Level:N', title='Nível Educacional'),
    alt.Y('Children:N', title='Intençao de ter filhos'),
    color='count():Q'
)
escolaridade_filhos


Nesse caso foi interessante a escolha de um mapa de calor, para explicitar, com a contagem no canal de cor, onde está a maior concentração em cada caso.

Agora podemos pensar em melhorias para a visualização desse gráfico, como traduzir os títulos, arrumar a escala, redimensionar e traduzir por meio do `tranform_calculate`.

In [ ]:
escolaridade_filhos = alt.Chart(df_app_namoro).transform_calculate(
        escolaridade="datum['Education Level'] === 'Ph.D.' ? 'Doutorado' : "
                     "datum['Education Level'] === 'Master\\'s Degree' ? 'Mestrado' : "
                     "datum['Education Level'] === 'Bachelor\\'s Degree' ? 'Bacharelado' : "
                     "'Ensino Médio'",
        filhos="datum.Children === 'Yes' ? 'Sim' : datum.Children === 'No' ? 'Não' : 'Talvez'"
    ).mark_rect().encode(
    alt.X('escolaridade:N', title='Nível Educacional', sort=["Ensino Médio", "Bacharelado", "Mestrado", 'Doutorado']),
    alt.Y('filhos:N', title='Intençao de ter filhos', sort=["Sim", "Talvez", "Não"]),
    color=alt.Color(
        'count():Q',
        scale=alt.Scale(
            type='log',
            range=['green', 'yellow', 'red']
        ),
        legend=alt.Legend(
            title='Contagem de Entradas',
            gradientLength=300,
            gradientThickness=20
        )
    ),
    tooltip=['count()',"escolaridade:N", 'filhos:N']
).properties(
    width=400,
    height=400,
    title='Nível Educacional vs. Intençao de ter filhos'
)

escolaridade_filhos


Agora com uma visualização clara, podemos entender com facilidade do que se trata o gráfico e os olhos vão diretamente nos pontos de interesse, as casas com mais concentração e as com menos.

#### Gráfico de exploração da base

A ideia inicial, por se tratar de uma base de 500 usuários, é tentar visualizar todos os usuários com seus respectivos atributos.

In [ ]:
painel_usuarios = alt.Chart(df_app_namoro).mark_circle().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive()

painel_usuarios


É perceptível que o gráfico não nos traz muita informação de cara, para isso pensei em contruir filtros, para que o leitor possa navegar pela base procurando o que quiser de maneira visual através do gráfico.

In [ ]:
input_dropdown = alt.binding_select(options=['Male', 'Female'], name='Gênero  ')
selectgênero = alt.selection_point(fields=['Gender'], bind=input_dropdown)
color = (
    alt.when(selectgênero)
    .then(alt.Color("Gender:N").legend(None))
    .otherwise(alt.value("lightgray"))
)


painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=color,
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(selectgênero)

painel_usuarios



Podemos então como visto nesse gráfico, codificar o gênero no canal cor, adicionar um botão interativo para que o leitor possa escolher o que visualizar alternamente.

Essa opção é interessante, mas já pensando em adicionar mais filtros, podemos trocar para um botão que não destaca a informação e sim mostra apenas ela.

In [ ]:
options = ['Male','Female']
labels = [option + ' ' for option in options]

input_dropdown = alt.binding_radio(
    options=options + [None],
    labels=labels + ['Both'],
    name='Gênero '
)
selection = alt.selection_point(
    fields=['Gender'],
    bind=input_dropdown,
)


painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=alt.Color('Gender:N').scale(domain=options),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(
    selection
).transform_filter(
    selection
)

painel_usuarios

Agora podemos adicionar `sliders` para permitir o leitor selecionar um limite de idade que deseja visualizar.

In [ ]:
slider_altura_min = alt.binding_range(min=146, max=190, step=0.5, name='Altura mínima ')
slider_altura_max = alt.binding_range(min=146, max=190, step=0.5, name='Altura máxima ')

selectoraltura_min = alt.param(name='altura_min', value=146, bind=slider_altura_min)
selectoraltura_max = alt.param(name='altura_max', value=190, bind=slider_altura_max)

predicate = (alt.datum.altura < selectoraltura_min) | (alt.datum.altura > selectoraltura_max)

painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=alt.when(predicate).then(alt.value("lightgray")).otherwise(alt.value("blue")),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(
    selection,
    selectoraltura_min,
    selectoraltura_max
).transform_filter(
    selection
)

painel_usuarios


De maneira análoga, adicionamos `sliders` para o intervalo de idade.

In [ ]:
botao_idade_min = alt.binding_range(min=17, max=36, step=1, name='Idade mínima ')
botao_idade_max = alt.binding_range(min=17, max=36, step=1, name='Idade máxima ')

selectoridade_min = alt.param(name='idade_min', value=17, bind=botao_idade_min)
selectoridade_max = alt.param(name='idade_max', value=36, bind=botao_idade_max)

predicate = ((alt.datum.altura < selectoraltura_min) | (alt.datum.altura > selectoraltura_max) |
             (alt.datum.Age < selectoridade_min) | (alt.datum.Age > selectoridade_max))

painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=alt.when(predicate).then(alt.value("lightgray")).otherwise(alt.value("blue")),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(
    selection,
    selectoraltura_min,
    selectoraltura_max,
    selectoridade_min,
    selectoridade_max
).transform_filter(
    selection
)

painel_usuarios


Para finalizar e darmos uma facilidade ainda maior para o leitor visualizar a informação da base, adicionamos botões que destacam a escolaridade e interesse solicitados. Os pontos em destaque são apenas aqueles que satisfazem todas as opções do leitor. E por último, acrescentamos título e legendas.

In [ ]:
menu_education = alt.binding_select(
    options=["Any", "High School", "Bachelor's Degree", "Master's Degree", "Ph.D."],
    name='Escolaridade '
)
selector_education = alt.param(name='education_min', value="Any", bind=menu_education)

menu_looking_for = alt.binding_select(
    options=["Any", "Casual Dating", "Friendship", "Marriage", "Long-term Relationship"],
    name='Interesse '
)
selector_looking_for = alt.param(name='looking_for', value="Any", bind=menu_looking_for)

predicate = (
    (alt.datum.altura < selectoraltura_min) | (alt.datum.altura > selectoraltura_max) |
    (alt.datum.Age < selectoridade_min) | (alt.datum.Age > selectoridade_max) |
    ((selector_education != "Any") & (alt.datum["Education Level"] != selector_education)) |
    ((selector_looking_for != "Any") & (alt.datum["Looking For"] != selector_looking_for))
)

painel_usuarios = (
    alt.Chart(df_app_namoro)
    .mark_point()
    .transform_calculate(altura='datum.Height * 30.48')
    .encode(
        alt.X('Age:Q', scale=alt.Scale(domain=[17, 36]), title="Idade"),
        alt.Y('altura:Q', scale=alt.Scale(domain=[146, 190]), title="Altura (cm)"),
        color=alt.condition(
            predicate,
            alt.value("lightgrey"),
            alt.value("blue")
        ),
        tooltip=['Age', 'altura:Q', 'Education Level', 'Gender', 'Looking For']
    )
    .properties(
        title="Painel Interativo de Usuários",
        width = 400,
        height = 400
    )
    .add_params(
        selection,
        selectoraltura_min,
        selectoraltura_max,
        selectoridade_min,
        selectoridade_max,
        selector_education,
        selector_looking_for
    ).transform_filter(
    selection
)
    .interactive()
)

painel_usuarios

## Felipe

### Análise de Dados - Conjunto de Dados *Wheat* do *Vega Datasets*

### Introdução
Neste estudo, utilizamos a biblioteca `altair` para realizar uma análise exploratória do conjunto de dados `wheat` do *Vega Datasets*. O objetivo é visualizar a distribuição dos tipos de trigo, a evolução dos salários ao longo dos anos e a relação entre salário e ano.

### Carregamento dos Dados
O conjunto de dados foi carregado utilizando:

In [ ]:
import altair as alt
from vega_datasets import data
import pandas as pd

# Carregar o conjunto de dados
wheat = data.wheat()

# Exibir as primeiras linhas
display(wheat.head())





Isso permite uma primeira inspeção dos dados e garante que estamos lidando corretamente com o *dataset*.

### Gráfico 1: Distribuição dos Tipos de Trigo
Para analisar a distribuição dos diferentes tipos de trigo no conjunto de dados, utilizamos um gráfico de barras:


In [ ]:
# Gráfico 1: Distribuição dos Tipos de Trigo
chart1 = alt.Chart(wheat).mark_bar().encode(
    x=alt.X('wheat:O', title='Tipo de Trigo'),
    y=alt.Y('count()', title='Quantidade'),
    color=alt.Color('wheat:N', legend=None),
    tooltip=['wheat', 'count()']
).properties(
    title='Distribuição dos Tipos de Trigo',
    width=600,
    height=400
).interactive()

chart1

### Interpretação
- O gráfico exibe a contagem de cada tipo de trigo presente no conjunto de dados.
- A visualização permite identificar quais tipos são mais frequentes.
- A interatividade ajuda na análise detalhada ao passar o cursor sobre as barras.

### Gráfico 2: Relação entre Salário e Ano
Para entender como os salários evoluíram ao longo dos anos para diferentes tipos de trigo, usamos um gráfico de linha:

In [ ]:
# Gráfico 2: Relação entre Rendimento e Ano da Colheita
chart2 = alt.Chart(wheat).mark_line(point=True).encode(
    x=alt.X('year:T', title='Ano'),
    y=alt.Y('wages:Q', title='Salário'),
    color=alt.Color('wheat:N', title='Tipo de Trigo'),
    tooltip=['year', 'wages', 'wheat']
).properties(
    title='Salário ao Longo dos Anos',
    width=700,
    height=400
).interactive()

chart2




### Interpretação
- Cada linha representa a variação do salário ao longo do tempo para um determinado tipo de trigo.
- É possível observar tendências de crescimento ou queda no salário ao longo dos anos.
- Os pontos destacados no gráfico indicam os valores individuais de cada ano.

### Gráfico 3: Comparação entre Salário e Ano
Para verificar melhor a dispersão dos salários ao longo do tempo, utilizamos um gráfico de dispersão (*scatter plot*):


In [ ]:
# Gráfico 3: Comparação entre Salário e Ano
chart3 = alt.Chart(wheat).mark_circle(size=80).encode(
    x=alt.X('year:T', title='Ano'),
    y=alt.Y('wages:Q', title='Salário'),
    color=alt.Color('wheat:N', title='Tipo de Trigo'),
    tooltip=['year', 'wages', 'wheat']
).properties(
    title='Relação entre Ano e Salário',
    width=700,
    height=400
).interactive()

chart3


### Interpretação
- Cada ponto representa o salário de um tipo de trigo em um determinado ano.
- A distribuição dos pontos pode indicar a existência de padrões, como aumento ou queda de salário ao longo do tempo.
- A cor representa o tipo de trigo, facilitando a comparação entre diferentes categorias.




### Estatísticas de Resumo
Para entender melhor as características do conjunto de dados, calculamos estatísticas descritivas:





In [ ]:
# Estatísticas descritivas gerais
descriptive_stats = wheat.describe()
display(descriptive_stats)

# Cálculo de frequências por tipo de trigo
frequency_counts = wheat['wheat'].value_counts()
display(frequency_counts)

### Interpretação
- O método `.describe()` fornece estatísticas como média, mediana, desvio padrão, valores mínimos e máximos.
- A contagem de frequências ajuda a visualizar a distribuição das categorias de trigo.
- Esses cálculos complementam as visualizações anteriores, fornecendo uma análise quantitativa detalhada.

### Conclusão
A análise permitiu:
1. Identificar a distribuição dos diferentes tipos de trigo.
2. Avaliar a evolução dos salários ao longo do tempo.
3. Visualizar a relação entre salário e ano para diferentes tipos de trigo.
4. Calcular estatísticas descritivas que ajudam a entender a distribuição e tendências dos dados.

Os gráficos interativos facilitaram a exploração dos dados e a identificação de padrões importantes. Outras análises poderiam incluir fatores adicionais, como a relação com a produção ou influências econômicas.


## Lucas Barros

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

Usaremos uma base de dados do site 'base dos dados' que trata aobre os jogos olimpicos modernos, tanto de verão quanto de inverno. O link para acesso da base de dados é https://basedosdados.org/dataset/a898f300-fa77-48dd-b4dd-59b83d7bb345?table=16d53ff3-afce-4c31-8a5c-bcb77a59078b


In [ ]:
dados = pd.read_csv('mundo_kaggle_olimpiadas_microdados.csv')
dados

Primeiramente, faremos uma limpeza nos dados deixando apenas os dados sobre as edições dos Jogos Olímpicos de Inverno.

In [ ]:
dados = dados.loc[dados['edicao'] == 'Winter']
dados

Agora, faremos uma primeira análise estatística sobre as alturas dos atletas em cada um dos esportes.

No início veremos as estatísticas descritivas das alturas de forma geral.

In [ ]:
dados['altura'].describe()

Agora faremos uma análise equivalente mas spearando esporte por esporte.

In [ ]:
dados.groupby('esporte')['altura'].describe()

A primeira coisa a se obserar é que não há dados sobre atletas de Patrulha Militar e nem de Alpinismo. Isso ocore decido ao fato de que a Patrulha Militar só foi de fato um evento de competição em 1924 e ocorreu outras 3 vezes como demonstração então por ser de fato disputado apenas na primeira edição os dados são mais escassos. Já sobre o Alpinismo temos uma situação parecidas, com mais participações mas todas extremamente antigas.

Além disso, podemos destacar agora que esporte tem a menor média de altura, no caso a patinação no gelo, e qual tem a maior média, no caso o bobsleigh. É possível também perceber quais esportes tem mais ou menos variação na altura, por exemplo, o combinado nórdico tem consideravelmente menos disperção na altura dos atletas que a patinação no gelo mesmo tendo uma média maior.

In [ ]:
dados.groupby(['esporte', 'medalha'])['altura'].describe()

Aqui percebemos uma diferença estatisticamente irrelevante ed medalhistas de ouro prata e bronze para determinado esporte.

 Após essas análises temos uma boa impressão sobre as alturas gerais dos atletas e dos atletas por esporte, no entanto ainda não é possível ter uma noção tão boa sobre a distribuição dessas alturas, portanto faremos uma visualização para ajudar nisso.

Antes será necessário desabilitar o número máximo de linhas do altair pois nossos dados tem mais de 5000 linhas.

In [ ]:
alt.data_transformers.disable_max_rows()


In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title='Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura'),
        alt.Tooltip('medalha')
        ]
).properties(
    title= 'Altura dos altetas'
)

Aqui já podemos perceber que há uma concentração em torno da média e não uma concentração tão grande nos extremos.

In [ ]:
alt.Chart(dados).mark_circle().transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title= 'Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura'),
        alt.Tooltip('medalha')
        ],
    color=alt.Color('medalha')
).properties(
    title= 'Altura dos altetas medalhistas'
)

Ja agora podemos perceber que entre os medalhistas isso também se aplica.

In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title= 'Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura')
        ]
).facet(
    facet='esporte:N',
    columns=3
)

Separando agora por esporte podemos ver que todos mantém o mesmo padrão tendo apenas algumas leves translações verticais

In [ ]:
alt.Chart(dados).mark_circle().transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title= 'Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura')
        ]
).facet(
    facet='medalha:N',
    columns=3
)

Aqui podemos perceber que esse comportamento se mantém ao também separando cada tipo de medalhista como era de se imaginar.

In [ ]:
alt.Chart(dados).mark_bar().encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 210]), title= 'Altura (cm)'),
    alt.Y('count()', title= 'Número de ocorrências'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura')
        ]
)

Por fim, adicionamos uma contagem para cada altura presente para podermos visualizar melhor as concentrações de atletas em barras e podemos perceber que se aproxima de uma distribuição normal.

Por fim, faremos um gráfico que mostra o ranking geral de todas as olímpiadas com uma parte interatva sendo possível "passear" pelos anos. Nesse caso cada medalha de ouro foi premiada com 6 pontos cada uma de prata com 3 pontos e as de bronze com 1 ponto.

In [ ]:
import altair as alt
import pandas as pd

slider = alt.binding_range(min=1924, max=2014, step=2, name='Ano')
slider_ano = alt.selection_point(
    name='slider',
    fields=['ano'],
    init={'ano': 1924},
    bind=slider,
    value=1924
)

# Define fixed country colors
unique_countries = dados['pais'].unique()
country_colors = alt.Scale(
    domain=unique_countries,
    range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
           '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
)

# Legend for Medal Points
legend_data = pd.DataFrame({
    'Medal': ['Ouro', 'Prata', 'Bronze'],
    'Points': [6, 3, 1]
})

legend = alt.Chart(legend_data).mark_rect().encode(
    y=alt.Y('Medal:N', title='Medalha', axis=alt.Axis(orient='left'), sort=['Ouro', 'Prata', 'Bronze']),
    color=alt.Color('Medal:N', scale=alt.Scale(
        domain=['Ouro', 'Prata', 'Bronze'],
        range=['gold', 'silver', 'brown']
    ), legend=None)
).properties(
    title='Pontos por Medalha',
    width=75,
    height=70
)

legend_text = alt.Chart(legend_data).mark_text(align='left').encode(
    y=alt.Y('Medal:N', title='Medalha', sort=['Ouro', 'Prata', 'Bronze'], axis=None),
    text=alt.Text('Points:N')
)

legend_chart = (legend + legend_text).resolve_scale(
    color='independent'
)

# Main Chart
chart = alt.Chart(dados).mark_bar().transform_calculate(
    pontos="{'Gold': 6, 'Silver': 3, 'Bronze': 1}[datum.medalha]"
).encode(
    alt.X('pais:N', title='Países', axis=alt.Axis(titleY=40)),
    alt.Y('sum(pontos):Q', scale=alt.Scale(domain=[0, 3600]), title='Soma dos pontos'),
    alt.Color('pais:N', scale=country_colors, legend=None)  # Fixed colors for countries
).transform_filter(
    alt.datum.ano <= slider_ano['ano']
).properties(
    width=700,
    height=500,
    title="Pontos relacionados à medalha de cada país nos Jogos Olímpicos de Inverno (Acumulativo)"
).add_params(
    slider_ano
)

# Final Layout
final_chart = alt.hconcat(
    chart,
    legend_chart
).configure_view(
    stroke=None
)

final_chart


Agora faremos outra visualização, dessa vez visando analisar os pesos e alturas de cada atleta.

Primeiro faremos novamente as análises estatísticas para as váriaveis de peso.


In [ ]:
dados['peso'].describe()

Temos algumas estatísticas mas seriam melhores se fossem separadas por esporte já que existem esportes que naturalmente favorecem pessoas mais pesadas ou mais leves.

In [ ]:
dados.groupby('esporte')['peso'].describe()

Percebeos que assim como na altura o Alpinismo e Patrulha Militar não tem dados e o motivo é o mesmo citado anteriormente.

Ademais, percebe-se que os esportes que tiveram maior média de alturas tendem a ter maiores pesos também, o que é completamente válido já que pessoas maiores tendem a pesar mais em média. Contudo, aida são precisas análises mais detalhadas para observar a real distribuição de pesos e se há alguma anormalidade inesperada.

Faremos agora uma análise, similar com a feita com as alturas, que abordará mais fortemente o quesito de medalhas ganhas pelos atletas.

In [ ]:
dados.groupby(['esporte', 'medalha'])['peso'].describe()

Após ver esses dados é claro observar que dado um determinado esporte as diferenças médias de pesos são estatisticamente insgnificantes visto o desvio padrão.

Para conseguirmos enfim ter certeza sobre co o é essa distribuição de pesos dos atletas faremos uma visualização.

In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso'),
        alt.Tooltip('medalha')
        ]
).properties(
    title= 'Peso dos altetas'
)

Aqui já podemos perceber que a distribuição se aproxima da de altura em certo sentido mas por outro lado parece ter uma concentração entorno da média mas um desvio padrão maior como as estatísticas indicavam.

In [ ]:
alt.Chart(dados).mark_circle().transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso'),
        alt.Tooltip('medalha')
        ],
    color=alt.Color('medalha')
).properties(
    title= 'Peso dos altetas medalhistas'
)

Os atletas medalhistas aparentam ter a mesma distribuição.'

In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso')
        ]
).facet(
    facet='esporte:N',
    columns=3
)

Com a separação por esportes já consegiumos ver uma diferenciação mais acentuada que na altura cmoo a diferença de patinação no gelo e bobsleigh

In [ ]:
alt.Chart(dados).mark_bar().encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    alt.Y('count()', title= 'Número de ocorreências'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso')
        ]
)

Com essa visualização podemos perceber uma espécie de distribuição normal com mais "outliers" para o lado direito e concentração maior na parte esquerda.

In [ ]:
chart2 = alt.Chart(dados).mark_point(filled=True).transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('altura', scale=alt.Scale(domain=[140, 205]), title='Altura (cm)'),
    alt.Y('peso', scale=alt.Scale(domain=[30, 130]), title='Peso (kg)'),
    alt.Column('esporte', title='Esporte'),
    color=alt.Color('medalha', title='Medalha'
    ),
    tooltip=[
        alt.Tooltip('nome_atleta', title='Atleta'),
        alt.Tooltip('medalha', title='Medalha'),
    ]
).properties(
    title='Altura x Peso dos atletas medalhistas (usando window ranking)'
)

chart2


Essa visualização nos permite observar um comportamento próximo do linear associando a altura com o peso e como esse comportamento linear se dá em cada esporte diferente, mas não nos permite ter boa visualização de alguma possíve distribuição dos medalhistas

In [ ]:
chart_fixed = alt.Chart(dados).mark_point(filled=True).transform_filter(
    alt.datum.medalha != None
).transform_calculate(
    esporte_pt="""{'Alpine Skiing': 'Esqui Alpino',
        'Biathlon': 'Biatlo',
        'Bobsleigh': 'Bobsleigh',
        'Cross Country Skiing': 'Esqui Cross Country',
        'Curling': 'Curling',
        'Figure Skating': 'Patinação Artística',
        'Freestyle Skiing': 'Esqui Estilo Livre',
        'Ice Hockey': 'Hóquei no gelo',
        'Luge': 'Luge',
        'Nordic Combined': 'Combinado Nórdico',
        'Short Track Speed Skating': 'Patinação de velocidade curta',
        'Skeleton': 'Skeleton',
        'Ski Jumping': 'Salto de Esqui',
        'Snowboarding': 'Snowboarding',
        'Speed Skating': 'Patinação de velocidade'
    }[datum.esporte]"""
).transform_calculate(
    medalha_valor="if(datum.medalha == 'Gold', 3, if(datum.medalha == 'Silver', 2, if(datum.medalha == 'Bronze', 1, 0)))"
).transform_joinaggregate(
    max_medalha_valor="max(medalha_valor)",
    groupby=["nome_atleta"]
).transform_filter(
    alt.datum.medalha_valor == alt.datum.max_medalha_valor
).encode(
    alt.X('altura', scale=alt.Scale(domain=[140, 205]), title='Altura (cm)'),
    alt.Y('peso', scale=alt.Scale(domain=[30, 130]), title='Peso (kg)'),
    alt.Column('esporte_pt:N', title='Esporte'),
    color=alt.Color('medalha:N', title='Medalha', scale=alt.Scale(
        domain=['Gold', 'Silver', 'Bronze'],
        range=['#FFD700', '#C0C0C0', '#CD7F32']
    ), legend=alt.Legend(
        titleFontSize=14,
        labelFontSize=12,
        symbolSize=200,
        orient='left',
        labelExpr="{'Gold': 'Ouro', 'Silver': 'Prata', 'Bronze': 'Bronze'}[datum.label]"
    )),
    tooltip=[
        alt.Tooltip('nome_atleta', title='Atleta'),
        alt.Tooltip('medalha', title='Medalha'),
        alt.Tooltip('medalha_valor:Q', title='Medalha Valor'),
        alt.Tooltip('max_medalha_valor:Q', title='Max Medalha Valor'),
        alt.Tooltip('esporte', title='esporte')
    ]
).properties(
    title='Altura x Peso dos atletas medalhistas (melhor medalha por atleta)'
)

chart_fixed


Agora podemos ter melhor noção de como se distribuem os medalhistas de ouro prata e bronze em cada esporte e podemos perceber que, por exemplo no esporte hóquei no gelo medalhistas de ouro tendem a ter uma estatura e peso maior que é um esporte extremamente físico mas já no curling percebemos um espaçamento maior já que é um esporte que exige menos do condicionamento físico.

Por fim pudemos perceber que os esportes têm grande influência sobre as medidas físicas de seus atletas, espcialmente de seus medalhista, inclusive quais tipos de medalhistas.

## Lucas Paulo Gonçalves

Farei dois gráficos usando o dataset cars, do vega_datasets.

Começo importando as bibliotecas necessárias e selecionando o dataset:

In [ ]:
import altair as alt
import pandas as pd
import vega_datasets as vdts

cars = vdts.data.cars()

In [ ]:
cars.head()

Para este primeiro gráfico, quero ver relações entre volume do motor (Displacement) e aceleração (Acceleration).

Estas são as estatísticas de resumo dos dois campos:

In [ ]:
print(cars[['Displacement', 'Acceleration']].describe(), '\n', cars['Origin'].mode())

Começo fazendo um plot simples dos dados:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration')
    )

Para este gráfico, também quero ver as origens de cada carro. Como neste dataset tenho apenas 3 origens possíveis (USA, Europe, Japan), usarei uma seleção de cores para diferenciar cada um:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration'),
    color = alt.Color('Origin:N')
    )

Para complementar, altero as cores para cores que julgo mais adequadas para esta divisão:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration'),
    color = alt.Color('Origin:N',
                       scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
    )
    )

Ter a legenda à direita não me agrada, então a movo para cima:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration'),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(orient='top'))
    )

Como todos os nomes estão em inglês, adicionarei títulos ao gráfico, à legenda e a cada eixo:

In [ ]:
alt.Chart(cars, title = 'Aceleração do motor por volume').mark_circle().encode(
    alt.X('Displacement:Q', title = 'Volume do motor (pol³)'),
    alt.Y('Acceleration:Q', title = 'Aceleração do motor (mph/s)'),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
    )

Normalmente, quão menor um motor é em volume, melhor ele performa, e quão maior a aceleração de um carro, melhor. Seguindo a ideia de que o melhor fica para cima/à direita, inverto a ordem do eixo X para que os motores menores em volume fiquem mais à direita:

In [ ]:
alt.Chart(cars, title = 'Aceleração do motor por volume').mark_circle().encode(
    alt.X('Displacement:Q', title = 'Volume do motor (pol³)', sort = 'descending'),
    alt.Y('Acceleration:Q', title = 'Aceleração do motor (mph/s)'),
    color = alt.Color('Origin:N',
                       scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
    )

Para finalizar, elimino os espaços brancos  para que possa melhor visualizar meus dados no gráfico. Para isso, tiro os intervalos que não contém pontos definindo o domínio de cada eixo:

In [ ]:
alt.Chart(cars, title = 'Aceleração do motor por volume').mark_circle().encode(
    alt.X('Displacement:Q', title = 'Volume do motor (pol³)', sort = 'descending', scale = alt.Scale(domain = [60, 470])),
    alt.Y('Acceleration:Q', title = 'Aceleração do motor (mph/s)', scale = alt.Scale(domain = [7, 26])),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
    )

Neste gráfico, é possível ver que os pontos são bem mais densos na região do volume entre 200 e 100 pol³. Faço uma análise unidimensional por meio de um histograma para confirmar este fato:

In [ ]:
alt.Chart(cars, title = 'Quantidade de carros por volume do motor').mark_bar().encode(
    alt.X('Displacement', title = 'Volume do motor (pol³)', sort = 'descending', bin = True),
    alt.Y('count()', title = 'Frequência')
)

Um fato que podemos obter visualmente no gráfico é que, em média, quão menor em volume o motor é, maior a aceleração. Podemos associar essa variação em aceleração ao peso do carro - quão mais leve o carro, maior deveria ser sua aceleração. Queremos, então associar o volume do motor ao peso do carro - quão maior o volume, maior o peso. Posso fazer isso numa análise bidimensional das duas variáveis, peso (Weight_in_lbs) e volume do motor (Displacement).

Para isso, farei um gráfico simples de pontos do peso pelo volume do motor:

In [ ]:
alt.Chart(cars, title = 'Peso dos carros por volume do motor').mark_circle().encode(
    alt.X('Displacement', title = 'Volume do motor (pol³)', sort = 'descending'),
    alt.Y('Weight_in_lbs', title = 'Peso (lbs)')
)

 Neste gráfico, já conseguimos ver uma relação clara entre volume do motor e peso. Para confirmar, faço a matriz de correlação desses dois campos de dados:

In [ ]:
cars[['Displacement', 'Weight_in_lbs']].corr()

O coeficiente de correlação do peso com o volume do motor é de 0.932, o que mostra uma relação linear entre peso do carro e volume do motor.

Gostaria de ver agora, no segundo gráfico, uma relação entre data de produção e peso.

Estas são as estatísticas de resumo:

In [ ]:
cars[['Year', 'Weight_in_lbs']].describe()

Começo fazendo um plot básico:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Year:T'),
    alt.Y('Weight_in_lbs')
)

Adiciono uma seleção de cor baseada na origem do carro, do mesmo esquema do gráfico anterior:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Year:T'),
    alt.Y('Weight_in_lbs'),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
)

Adiciono títulos aos eixos e ao gráfico e limito o domínio do peso, já que este tem mínimo em 1613:

In [ ]:
alt.Chart(cars, title = 'Peso do carro por ano').mark_circle().encode(
    alt.X('Year:T', title = 'Ano de produção'),
    alt.Y('Weight_in_lbs', title = 'Peso (lbs)', scale = alt.Scale(domain = [1600, 5200])),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
)

Este gráfico já está útil, porém quero visualizar melhor os pontos na parte inferior, logo vou aplicar uma escala logarítmica de base 10 no eixo Y:

In [ ]:
alt.Chart(cars, title = 'Peso do carro por ano').mark_circle().encode(
    alt.X('Year:T', title = 'Ano de produção'),
    alt.Y('Weight_in_lbs', title = 'Peso (lbs)', scale = alt.Scale(domain = [1600, 5200], type = 'log')),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
)

Com este gráfico, podemos ver e afirmar que os carros americanos começaram consistentemente mais pesados que os carros europeus e japoneses, porem, com o passar do tempo, tem alcançado medidas similares. Isso explica o comportamento visto no primeiro gráfico - carros com maior volume de motor (mais pesados) são predominantemente americanos, porém são definitivamente mais antigos. Faço uma análise unidimensional dos carros americanos para confirmar que existe uma quantidade considerável de carros americanos em todas as classes de peso:

In [ ]:
alt.Chart(cars[cars['Origin'] == 'USA'], title = 'Histograma de frequência de carros americanos por classe de peso').mark_bar().encode(
    alt.X('Weight_in_lbs', title = 'Peso (lbs)', bin = True),
    alt.Y('count()', title = 'Frequência')
)

Podemos ver que carros americanos tem um peso bem distribuído, o que pode justificar a grande dispersão dos pontos americanos no gráfico 1. Para finalizar a lógica, farei uma análise bidimensional de carros americanos baseado na kilometragem do carro (`Horsepower`) e o volume do motor:

In [ ]:
alt.Chart(cars[cars['Origin'] == 'USA'], title = 'Potência em cavalos por volume do motor').mark_circle().encode(
    alt.X('Displacement', title = 'Volume do motor (m³)'),
    alt.Y('Miles_per_Gallon', title = 'Kilometragem (mpg)')
)

A partir desses dois gráficos gerados, podemos chegar a uma conjectura: Carros americanos vem evoluindo com o tempo. Eram originalmente bem mais pesados em comparação aos europeus e japoneses, mas vem consistentemente se tornando mais leves. Essa diminuição de peso é um grande sinal de aumento de qualidade, já que carros mais leves tem motores menores, que por sua vez aceleram mais e bebem menos, o que quer dizer menor consumo de combustível e logo menos poluição! Logo, a partir apenas desses dados, podemos dizer que carros americanos vem se tornando menos poluentes com o passar dos anos!

# Gráficos Individuais

## Carlos

Primeiramente vamos importar as bibliotecas necessárias

In [ ]:
import altair as alt
import pandas as pd


Para nossos estudos, utilizaremos uma base de dados obtida no site [base dos dados](https://basedosdados.org/). Foi escolhido uma base que contempla dados do saneamanto básico de Pernambuco.

In [ ]:
file_id = "12j_IuZAzITdIZPGvamLQbYaNo-YvwqKU"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

alt.data_transformers.disable_max_rows()

df = pd.read_csv(url)
df.head()


Vamos observar, primeiramente, como se da a distrubuição de água.

In [ ]:
alt.Chart(df).mark_boxplot().encode(
    y=alt.Y('populacao_urbana_atendida_agua:Q', title='População Urbana Atendida com Água')
).properties(
    title='Boxplot da População Urbana Atendida com Água'
)

Perceba que esse gráfico não diz muita coisa, na verdade ele não nos representa nada, visto que a distrubuição de água por cidade e o número de residentes não é levando em conta.

Vamos observar a distribuição de água de pernambuco no geral no decorrer dos anos.

In [ ]:
numero_agua_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),

).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_agua_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana_atendida_agua):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_agua_line + numero_agua_pontos

Podemos observar aparente crescimento no número de pessoas que receberam água durante. Porém isso não confirma nada, visto que a população também cresce ao longo do tempo. Portanto, para uma observação honesta, vamos plotar junto o crescimento da população urbana a cada ano. Além do mais, perceba que não temos dados do ano de 2000 para trás. Então vamos retirar esses dados


In [ ]:
df = df[df['ano'] > 2000]

numero_agua_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),

).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_agua_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_agua):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana_atendida_agua):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)


numero_pessoas_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_pessoas_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)

numero_agua_line + numero_agua_pontos + numero_pessoas_pontos + numero_pessoas_line

Aqui percebemos eventos estranhos no dado, como a diminuição repentina de pessoas com água em casa sem ter diminuição na população no ano de 2010. É claro, essa diminuição repentina pode ser simplesmente um erro no nosso dataframe.

Vamos observar o caso dos esgotos.

In [ ]:

numero_esgoto_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_esgoto):Q', title='Número de Pessoas Atendidas com Água'),

).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_esgoto_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana_atendida_esgoto):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana_atendida_esgoto):Q')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)


numero_pessoas_line =alt.Chart(df).mark_line().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água')
).properties(
    title='Número de Pessoas Atendidas com Água por Ano'
)
numero_pessoas_pontos =alt.Chart(df).mark_point().encode(
    alt.X('ano:O', title='Ano'),
    alt.Y('sum(populacao_urbana):Q', title='Número de Pessoas Atendidas com Água'),
    alt.Tooltip('sum(populacao_urbana):Q')
).properties(
    title='Número de Pessoas Atendidas com Esgoto por Ano'
)

numero_esgoto_line + numero_esgoto_pontos + numero_pessoas_pontos + numero_pessoas_line

Aqui observamos a situação mais crírica apresentada. O número de pessoas com esgoto (pelo menos o que a pesquisa entende como esgoto) é menor do que o número de residentes.

Porém, todos esse gráficos não nos dão uma pespectiva em percential. Vamos observar a caracteristicas percertuais de cada cidade.

In [ ]:
df["proporcao_agua"] = df["populacao_urbana_atendida_agua"] / df["populacao_urbana"]
df["proporcao_esgoto"] = df["populacao_urbana_atendida_esgoto"] / df["populacao_urbana"]


df_long = df.melt(id_vars=["ano", "id_municipio"],
                  value_vars=["proporcao_agua", "proporcao_esgoto"],
                  var_name="Serviço", value_name="Proporção")

selecao_ano = alt.param(
    name="ano_escolhido",
    bind=alt.binding_select(options=sorted(df["ano"].unique()), name="Ano: "),
    value=df["ano"].min()
)
agua = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida", scale=alt.Scale(domain=[0, 1.4])),
    color=alt.value("blue"),
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".2f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_agua") & (alt.datum.ano == selecao_ano)
)

esgoto = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida"),
    color=alt.value("green"),
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".2f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_esgoto") & (alt.datum.ano == selecao_ano)
)


grafico = (agua + esgoto).add_params(selecao_ano).properties(
    title="Cobertura de Água e Esgoto nos Municípios",
    width=1200,
    height=800
)

grafico

Observe que muitas cidades em vários anos não tem nada de esgoto. Isso parece muito estanho, muito provavelmente exitem várias colunas na base de dados onde não temos informação alguma sobre esgotos na cidade. Vamos mudar o dados e remover as linhas sem iformação de esgoto usando o método `dropna`.

In [ ]:
df = df.dropna(subset=["populacao_urbana_atendida_esgoto"])


df["proporcao_agua"] = df["populacao_urbana_atendida_agua"] / df["populacao_urbana"]
df["proporcao_esgoto"] = df["populacao_urbana_atendida_esgoto"] / df["populacao_urbana"]


df_long = df.melt(id_vars=["ano", "id_municipio"],
                  value_vars=["proporcao_agua", "proporcao_esgoto"],
                  var_name="Serviço", value_name="Proporção")

selecao_ano = alt.param(
    name="ano_escolhido",
    bind=alt.binding_select(options=sorted(df["ano"].unique()), name="Ano: "),
    value=df["ano"].min()
)


agua = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida (%)", scale=alt.Scale(domain=[0, 1.4])),
    color=alt.value("blue"),  # Define a cor fixa para água
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".2f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_agua") & (alt.datum.ano == selecao_ano)
)

esgoto = alt.Chart(df_long).mark_bar(opacity=0.6).encode(
    x=alt.X("id_municipio:N", title="Município (ID)"),
    y=alt.Y("Proporção:Q", title="Proporção da População Atendida (%)"),
    color=alt.value("green"),  # Define a cor fixa para esgoto
    tooltip=["id_municipio", "Serviço", alt.Tooltip("Proporção:Q", format=".1f")]
).transform_filter(
    (alt.datum.Serviço == "proporcao_esgoto") & (alt.datum.ano == selecao_ano)
)

# Combina as duas camadas no mesmo gráfico
grafico = (agua + esgoto).add_params(selecao_ano).properties(
    title="Cobertura de Água e Esgoto nos Municípios (%)",
    width=1200,
    height=800
)

grafico

## Daniel Couto

In [ ]:
import altair as alt
import pandas as pd

Para a minha análise utilizarei uma base do Kaggle, com os dados de usuários de um determinado aplicativo de relacionamento. A base possui colunas como: Idade, gênero, altura, objetivo da pessoa no app e etc.

In [ ]:
df_app_namoro = pd.read_csv('app_namoro.csv')
# Link para a base de dados: https://www.kaggle.com/datasets/anandshaw2001/dating-dataset?resource=download
df_app_namoro = pd.DataFrame(df_app_namoro)
df_app_namoro.head()


#### Estatísticas de resumo

Vamos agora calcular medidas de resumo de algumas das variveis do DataFrame. E para uma visão geral das variáveis quantitativas podemos usar também o `.describe()`.

In [ ]:
df_app_namoro.describe()

In [ ]:
import numpy as np
media_idade = np.mean(df_app_namoro['Age'])
desvio_idade = np.std(df_app_namoro['Age'])
print(f'Média idade = {media_idade:.1f} anos'.replace('.',','))
print(f'Desvio padrão das idades = {desvio_idade:.1f} anos'.replace('.',','))
print()

media_altura = np.mean(df_app_namoro['Height'])*0.3048 #Usando a altura em metros
altura_max = max(df_app_namoro['Height'])*0.3048
desvio_altura = np.std(df_app_namoro['Height'])*30.48 #Usando a altura em centimetros
print(f'Média altura = {media_altura:.2f} m'.replace('.',','))
print(f'Maior altura = {altura_max:.2f} m'.replace('.',','))
print(f'Desvio padrão das alturas = {desvio_altura:.1f} cm'.replace('.',','))
print()

media_altura_homem = np.mean(df_app_namoro.loc[df_app_namoro['Gender'] == "Male"]['Height'])*0.3048
media_altura_mulher = np.mean(df_app_namoro.loc[df_app_namoro['Gender'] == "Female"]['Height'])*0.3048
print(f'Média altura dos homens = {media_altura_homem:.3f} m'.replace('.',','))
print(f'Média altura das mulheres = {media_altura_mulher:.3f} m'.replace('.',','))
print()

moda_interesses = df_app_namoro['Interests'].mode()
print(f'Conjunto de interesses modal: {moda_interesses}') # Essa maneira não calcula exatamente o interesse modal, e sim o conjunto de interesses modal
print()

# Para resolver isso, abrimos as listas em linhas separadas com cada interesses
interests_exploded = df_app_namoro.explode('Interests')
moda_interesse = interests_exploded['Interests'].mode()
print(f'Interesse modal: {moda_interesse}')
print()

filho = df_app_namoro['Children'].value_counts('Yes')
print(filho)

#### Análise Unidimensional

Vamos agora analisar a variável `Looking For` que nos diz o que o usuário procura no aplicativo.'

In [ ]:
objetivo = alt.Chart(df_app_namoro).mark_bar().encode(
    alt.X('Looking For'),
    alt.Y('count()'),
    color = 'Looking For'
)
objetivo

Um histograma foi a escolha mais intuitiva para uma informação de contagem e comparação entre categorias. E as cores apesar de estarem passando a mesma informação do eixo x, ajuda na visualização e entendimento.

Podemos fazer algumas melhorias no nosso gráfico, como botar títulos, traduzir a legenda, escolher as cores, redimensionar o gráfico e ainda separar entre homens e mulheres para compará-los.

In [ ]:
objetivo = alt.Chart(df_app_namoro).transform_calculate(
    traducao="datum['Looking For'] == 'Casual Dating' ? 'Encontro Casual' : "
             "datum['Looking For'] == 'Marriage' ? 'Casamento' : "
             "datum['Looking For'] == 'Long-term Relationship' ? 'Relacionamento Sério' : "
             "'Amizade'",
    genero_traduzido="datum.Gender === 'Female' ? 'Mulher' : 'Homem'"
).mark_bar().encode(
    alt.X('traducao:N', title="Interesses"),
    alt.Y('count()', title="Quantidade de Usuários"),
    alt.Color('traducao:N',
        scale=alt.Scale(
            domain=["Encontro Casual", "Casamento", "Relacionamento Sério", "Amizade"],
            range=["#eb584d", "#e0c641","#5c58e0" ,"#55e08f" ]
        ),
        legend=alt.Legend(title="Tipo de Interesse")
    ),
    column=alt.Column("genero_traduzido:N", header=alt.Header(title="Gênero"))
).properties(
    height = 450,
    width=225,
    title= alt.TitleParams(
        text="Procura dos usuários de app de relacionamento",
        anchor="middle")
)

objetivo


#### Análise Bidimensional

Nessa análise vamos comparar o nível de escolaridade com a intenção de ter filhos.

In [ ]:
escolaridade_filhos = alt.Chart(df_app_namoro).mark_rect().encode(
    alt.X('Education Level:N', title='Nível Educacional'),
    alt.Y('Children:N', title='Intençao de ter filhos'),
    color='count():Q'
)
escolaridade_filhos


Nesse caso foi interessante a escolha de um mapa de calor, para explicitar, com a contagem no canal de cor, onde está a maior concentração em cada caso.

Agora podemos pensar em melhorias para a visualização desse gráfico, como traduzir os títulos, arrumar a escala, redimensionar e traduzir por meio do `tranform_calculate`.

In [ ]:
escolaridade_filhos = alt.Chart(df_app_namoro).transform_calculate(
        escolaridade="datum['Education Level'] === 'Ph.D.' ? 'Doutorado' : "
                     "datum['Education Level'] === 'Master\\'s Degree' ? 'Mestrado' : "
                     "datum['Education Level'] === 'Bachelor\\'s Degree' ? 'Bacharelado' : "
                     "'Ensino Médio'",
        filhos="datum.Children === 'Yes' ? 'Sim' : datum.Children === 'No' ? 'Não' : 'Talvez'"
    ).mark_rect().encode(
    alt.X('escolaridade:N', title='Nível Educacional', sort=["Ensino Médio", "Bacharelado", "Mestrado", 'Doutorado']),
    alt.Y('filhos:N', title='Intençao de ter filhos', sort=["Sim", "Talvez", "Não"]),
    color=alt.Color(
        'count():Q',
        scale=alt.Scale(
            type='log',
            range=['green', 'yellow', 'red']
        ),
        legend=alt.Legend(
            title='Contagem de Entradas',
            gradientLength=300,
            gradientThickness=20
        )
    ),
    tooltip=['count()',"escolaridade:N", 'filhos:N']
).properties(
    width=400,
    height=400,
    title='Nível Educacional vs. Intençao de ter filhos'
)

escolaridade_filhos


Agora com uma visualização clara, podemos entender com facilidade do que se trata o gráfico e os olhos vão diretamente nos pontos de interesse, as casas com mais concentração e as com menos.

#### Gráfico de exploração da base

A ideia inicial, por se tratar de uma base de 500 usuários, é tentar visualizar todos os usuários com seus respectivos atributos.

In [ ]:
painel_usuarios = alt.Chart(df_app_namoro).mark_circle().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive()

painel_usuarios


É perceptível que o gráfico não nos traz muita informação de cara, para isso pensei em contruir filtros, para que o leitor possa navegar pela base procurando o que quiser de maneira visual através do gráfico.

In [ ]:
input_dropdown = alt.binding_select(options=['Male', 'Female'], name='Gênero  ')
selectgênero = alt.selection_point(fields=['Gender'], bind=input_dropdown)
color = (
    alt.when(selectgênero)
    .then(alt.Color("Gender:N").legend(None))
    .otherwise(alt.value("lightgray"))
)


painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=color,
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(selectgênero)

painel_usuarios



Podemos então como visto nesse gráfico, codificar o gênero no canal cor, adicionar um botão interativo para que o leitor possa escolher o que visualizar alternamente.

Essa opção é interessante, mas já pensando em adicionar mais filtros, podemos trocar para um botão que não destaca a informação e sim mostra apenas ela.

In [ ]:
options = ['Male','Female']
labels = [option + ' ' for option in options]

input_dropdown = alt.binding_radio(
    options=options + [None],
    labels=labels + ['Both'],
    name='Gênero '
)
selection = alt.selection_point(
    fields=['Gender'],
    bind=input_dropdown,
)


painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=alt.Color('Gender:N').scale(domain=options),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(
    selection
).transform_filter(
    selection
)

painel_usuarios

Agora podemos adicionar `sliders` para permitir o leitor selecionar um limite de idade que deseja visualizar.

In [ ]:
slider_altura_min = alt.binding_range(min=146, max=190, step=0.5, name='Altura mínima ')
slider_altura_max = alt.binding_range(min=146, max=190, step=0.5, name='Altura máxima ')

selectoraltura_min = alt.param(name='altura_min', value=146, bind=slider_altura_min)
selectoraltura_max = alt.param(name='altura_max', value=190, bind=slider_altura_max)

predicate = (alt.datum.altura < selectoraltura_min) | (alt.datum.altura > selectoraltura_max)

painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=alt.when(predicate).then(alt.value("lightgray")).otherwise(alt.value("blue")),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(
    selection,
    selectoraltura_min,
    selectoraltura_max
).transform_filter(
    selection
)

painel_usuarios


De maneira análoga, adicionamos `sliders` para o intervalo de idade.

In [ ]:
botao_idade_min = alt.binding_range(min=17, max=36, step=1, name='Idade mínima ')
botao_idade_max = alt.binding_range(min=17, max=36, step=1, name='Idade máxima ')

selectoridade_min = alt.param(name='idade_min', value=17, bind=botao_idade_min)
selectoridade_max = alt.param(name='idade_max', value=36, bind=botao_idade_max)

predicate = ((alt.datum.altura < selectoraltura_min) | (alt.datum.altura > selectoraltura_max) |
             (alt.datum.Age < selectoridade_min) | (alt.datum.Age > selectoridade_max))

painel_usuarios = alt.Chart(df_app_namoro).mark_point().transform_calculate(
    altura='datum.Height * 30.48'
).encode(
    alt.X('Age:Q', scale=alt.Scale(domain=[17,36])),
    alt.Y('altura:Q', scale=alt.Scale(domain=[146,190])),
    color=alt.when(predicate).then(alt.value("lightgray")).otherwise(alt.value("blue")),
    tooltip=['Age','altura:Q','Education Level','Gender']
).interactive().add_params(
    selection,
    selectoraltura_min,
    selectoraltura_max,
    selectoridade_min,
    selectoridade_max
).transform_filter(
    selection
)

painel_usuarios


Para finalizar e darmos uma facilidade ainda maior para o leitor visualizar a informação da base, adicionamos botões que destacam a escolaridade e interesse solicitados. Os pontos em destaque são apenas aqueles que satisfazem todas as opções do leitor. E por último, acrescentamos título e legendas.

In [ ]:
menu_education = alt.binding_select(
    options=["Any", "High School", "Bachelor's Degree", "Master's Degree", "Ph.D."],
    name='Escolaridade '
)
selector_education = alt.param(name='education_min', value="Any", bind=menu_education)

menu_looking_for = alt.binding_select(
    options=["Any", "Casual Dating", "Friendship", "Marriage", "Long-term Relationship"],
    name='Interesse '
)
selector_looking_for = alt.param(name='looking_for', value="Any", bind=menu_looking_for)

predicate = (
    (alt.datum.altura < selectoraltura_min) | (alt.datum.altura > selectoraltura_max) |
    (alt.datum.Age < selectoridade_min) | (alt.datum.Age > selectoridade_max) |
    ((selector_education != "Any") & (alt.datum["Education Level"] != selector_education)) |
    ((selector_looking_for != "Any") & (alt.datum["Looking For"] != selector_looking_for))
)

painel_usuarios = (
    alt.Chart(df_app_namoro)
    .mark_point()
    .transform_calculate(altura='datum.Height * 30.48')
    .encode(
        alt.X('Age:Q', scale=alt.Scale(domain=[17, 36]), title="Idade"),
        alt.Y('altura:Q', scale=alt.Scale(domain=[146, 190]), title="Altura (cm)"),
        color=alt.condition(
            predicate,
            alt.value("lightgrey"),
            alt.value("blue")
        ),
        tooltip=['Age', 'altura:Q', 'Education Level', 'Gender', 'Looking For']
    )
    .properties(
        title="Painel Interativo de Usuários",
        width = 400,
        height = 400
    )
    .add_params(
        selection,
        selectoraltura_min,
        selectoraltura_max,
        selectoridade_min,
        selectoridade_max,
        selector_education,
        selector_looking_for
    ).transform_filter(
    selection
)
    .interactive()
)

painel_usuarios

## Felipe

### Análise de Dados - Conjunto de Dados *Wheat* do *Vega Datasets*

### Introdução
Neste estudo, utilizamos a biblioteca `altair` para realizar uma análise exploratória do conjunto de dados `wheat` do *Vega Datasets*. O objetivo é visualizar a distribuição dos tipos de trigo, a evolução dos salários ao longo dos anos e a relação entre salário e ano.

### Carregamento dos Dados
O conjunto de dados foi carregado utilizando:

In [ ]:
import altair as alt
from vega_datasets import data
import pandas as pd

# Carregar o conjunto de dados
wheat = data.wheat()

# Exibir as primeiras linhas
display(wheat.head())





Isso permite uma primeira inspeção dos dados e garante que estamos lidando corretamente com o *dataset*.

### Gráfico 1: Distribuição dos Tipos de Trigo
Para analisar a distribuição dos diferentes tipos de trigo no conjunto de dados, utilizamos um gráfico de barras:


In [ ]:
# Gráfico 1: Distribuição dos Tipos de Trigo
chart1 = alt.Chart(wheat).mark_bar().encode(
    x=alt.X('wheat:O', title='Tipo de Trigo'),
    y=alt.Y('count()', title='Quantidade'),
    color=alt.Color('wheat:N', legend=None),
    tooltip=['wheat', 'count()']
).properties(
    title='Distribuição dos Tipos de Trigo',
    width=600,
    height=400
).interactive()

chart1

### Interpretação
- O gráfico exibe a contagem de cada tipo de trigo presente no conjunto de dados.
- A visualização permite identificar quais tipos são mais frequentes.
- A interatividade ajuda na análise detalhada ao passar o cursor sobre as barras.

### Gráfico 2: Relação entre Salário e Ano
Para entender como os salários evoluíram ao longo dos anos para diferentes tipos de trigo, usamos um gráfico de linha:

In [ ]:
# Gráfico 2: Relação entre Rendimento e Ano da Colheita
chart2 = alt.Chart(wheat).mark_line(point=True).encode(
    x=alt.X('year:T', title='Ano'),
    y=alt.Y('wages:Q', title='Salário'),
    color=alt.Color('wheat:N', title='Tipo de Trigo'),
    tooltip=['year', 'wages', 'wheat']
).properties(
    title='Salário ao Longo dos Anos',
    width=700,
    height=400
).interactive()

chart2




### Interpretação
- Cada linha representa a variação do salário ao longo do tempo para um determinado tipo de trigo.
- É possível observar tendências de crescimento ou queda no salário ao longo dos anos.
- Os pontos destacados no gráfico indicam os valores individuais de cada ano.

### Gráfico 3: Comparação entre Salário e Ano
Para verificar melhor a dispersão dos salários ao longo do tempo, utilizamos um gráfico de dispersão (*scatter plot*):


In [ ]:
# Gráfico 3: Comparação entre Salário e Ano
chart3 = alt.Chart(wheat).mark_circle(size=80).encode(
    x=alt.X('year:T', title='Ano'),
    y=alt.Y('wages:Q', title='Salário'),
    color=alt.Color('wheat:N', title='Tipo de Trigo'),
    tooltip=['year', 'wages', 'wheat']
).properties(
    title='Relação entre Ano e Salário',
    width=700,
    height=400
).interactive()

chart3


### Interpretação
- Cada ponto representa o salário de um tipo de trigo em um determinado ano.
- A distribuição dos pontos pode indicar a existência de padrões, como aumento ou queda de salário ao longo do tempo.
- A cor representa o tipo de trigo, facilitando a comparação entre diferentes categorias.




### Estatísticas de Resumo
Para entender melhor as características do conjunto de dados, calculamos estatísticas descritivas:





In [ ]:
# Estatísticas descritivas gerais
descriptive_stats = wheat.describe()
display(descriptive_stats)

# Cálculo de frequências por tipo de trigo
frequency_counts = wheat['wheat'].value_counts()
display(frequency_counts)

### Interpretação
- O método `.describe()` fornece estatísticas como média, mediana, desvio padrão, valores mínimos e máximos.
- A contagem de frequências ajuda a visualizar a distribuição das categorias de trigo.
- Esses cálculos complementam as visualizações anteriores, fornecendo uma análise quantitativa detalhada.

### Conclusão
A análise permitiu:
1. Identificar a distribuição dos diferentes tipos de trigo.
2. Avaliar a evolução dos salários ao longo do tempo.
3. Visualizar a relação entre salário e ano para diferentes tipos de trigo.
4. Calcular estatísticas descritivas que ajudam a entender a distribuição e tendências dos dados.

Os gráficos interativos facilitaram a exploração dos dados e a identificação de padrões importantes. Outras análises poderiam incluir fatores adicionais, como a relação com a produção ou influências econômicas.


## Lucas Barros

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

Usaremos uma base de dados do site 'base dos dados' que trata aobre os jogos olimpicos modernos, tanto de verão quanto de inverno. O link para acesso da base de dados é https://basedosdados.org/dataset/a898f300-fa77-48dd-b4dd-59b83d7bb345?table=16d53ff3-afce-4c31-8a5c-bcb77a59078b


In [ ]:
dados = pd.read_csv('mundo_kaggle_olimpiadas_microdados.csv')
dados

Primeiramente, faremos uma limpeza nos dados deixando apenas os dados sobre as edições dos Jogos Olímpicos de Inverno.

In [ ]:
dados = dados.loc[dados['edicao'] == 'Winter']
dados

Agora, faremos uma primeira análise estatística sobre as alturas dos atletas em cada um dos esportes.

No início veremos as estatísticas descritivas das alturas de forma geral.

In [ ]:
dados['altura'].describe()

Agora faremos uma análise equivalente mas spearando esporte por esporte.

In [ ]:
dados.groupby('esporte')['altura'].describe()

A primeira coisa a se obserar é que não há dados sobre atletas de Patrulha Militar e nem de Alpinismo. Isso ocore decido ao fato de que a Patrulha Militar só foi de fato um evento de competição em 1924 e ocorreu outras 3 vezes como demonstração então por ser de fato disputado apenas na primeira edição os dados são mais escassos. Já sobre o Alpinismo temos uma situação parecidas, com mais participações mas todas extremamente antigas.

Além disso, podemos destacar agora que esporte tem a menor média de altura, no caso a patinação no gelo, e qual tem a maior média, no caso o bobsleigh. É possível também perceber quais esportes tem mais ou menos variação na altura, por exemplo, o combinado nórdico tem consideravelmente menos disperção na altura dos atletas que a patinação no gelo mesmo tendo uma média maior.

In [ ]:
dados.groupby(['esporte', 'medalha'])['altura'].describe()

Aqui percebemos uma diferença estatisticamente irrelevante ed medalhistas de ouro prata e bronze para determinado esporte.

 Após essas análises temos uma boa impressão sobre as alturas gerais dos atletas e dos atletas por esporte, no entanto ainda não é possível ter uma noção tão boa sobre a distribuição dessas alturas, portanto faremos uma visualização para ajudar nisso.

Antes será necessário desabilitar o número máximo de linhas do altair pois nossos dados tem mais de 5000 linhas.

In [ ]:
alt.data_transformers.disable_max_rows()


In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title='Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura'),
        alt.Tooltip('medalha')
        ]
).properties(
    title= 'Altura dos altetas'
)

Aqui já podemos perceber que há uma concentração em torno da média e não uma concentração tão grande nos extremos.

In [ ]:
alt.Chart(dados).mark_circle().transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title= 'Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura'),
        alt.Tooltip('medalha')
        ],
    color=alt.Color('medalha')
).properties(
    title= 'Altura dos altetas medalhistas'
)

Ja agora podemos perceber que entre os medalhistas isso também se aplica.

In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title= 'Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura')
        ]
).facet(
    facet='esporte:N',
    columns=3
)

Separando agora por esporte podemos ver que todos mantém o mesmo padrão tendo apenas algumas leves translações verticais

In [ ]:
alt.Chart(dados).mark_circle().transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 215]), title= 'Altura (cm)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura')
        ]
).facet(
    facet='medalha:N',
    columns=3
)

Aqui podemos perceber que esse comportamento se mantém ao também separando cada tipo de medalhista como era de se imaginar.

In [ ]:
alt.Chart(dados).mark_bar().encode(
    alt.X('altura',
          scale=alt.Scale(domain=[130, 210]), title= 'Altura (cm)'),
    alt.Y('count()', title= 'Número de ocorrências'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('altura')
        ]
)

Por fim, adicionamos uma contagem para cada altura presente para podermos visualizar melhor as concentrações de atletas em barras e podemos perceber que se aproxima de uma distribuição normal.

Por fim, faremos um gráfico que mostra o ranking geral de todas as olímpiadas com uma parte interatva sendo possível "passear" pelos anos. Nesse caso cada medalha de ouro foi premiada com 6 pontos cada uma de prata com 3 pontos e as de bronze com 1 ponto.

In [ ]:
import altair as alt
import pandas as pd

slider = alt.binding_range(min=1924, max=2014, step=2, name='Ano')
slider_ano = alt.selection_point(
    name='slider',
    fields=['ano'],
    init={'ano': 1924},
    bind=slider,
    value=1924
)

# Define fixed country colors
unique_countries = dados['pais'].unique()
country_colors = alt.Scale(
    domain=unique_countries,
    range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
           '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
)

# Legend for Medal Points
legend_data = pd.DataFrame({
    'Medal': ['Ouro', 'Prata', 'Bronze'],
    'Points': [6, 3, 1]
})

legend = alt.Chart(legend_data).mark_rect().encode(
    y=alt.Y('Medal:N', title='Medalha', axis=alt.Axis(orient='left'), sort=['Ouro', 'Prata', 'Bronze']),
    color=alt.Color('Medal:N', scale=alt.Scale(
        domain=['Ouro', 'Prata', 'Bronze'],
        range=['gold', 'silver', 'brown']
    ), legend=None)
).properties(
    title='Pontos por Medalha',
    width=75,
    height=70
)

legend_text = alt.Chart(legend_data).mark_text(align='left').encode(
    y=alt.Y('Medal:N', title='Medalha', sort=['Ouro', 'Prata', 'Bronze'], axis=None),
    text=alt.Text('Points:N')
)

legend_chart = (legend + legend_text).resolve_scale(
    color='independent'
)

# Main Chart
chart = alt.Chart(dados).mark_bar().transform_calculate(
    pontos="{'Gold': 6, 'Silver': 3, 'Bronze': 1}[datum.medalha]"
).encode(
    alt.X('pais:N', title='Países', axis=alt.Axis(titleY=40)),
    alt.Y('sum(pontos):Q', scale=alt.Scale(domain=[0, 3600]), title='Soma dos pontos'),
    alt.Color('pais:N', scale=country_colors, legend=None)  # Fixed colors for countries
).transform_filter(
    alt.datum.ano <= slider_ano['ano']
).properties(
    width=700,
    height=500,
    title="Pontos relacionados à medalha de cada país nos Jogos Olímpicos de Inverno (Acumulativo)"
).add_params(
    slider_ano
)

# Final Layout
final_chart = alt.hconcat(
    chart,
    legend_chart
).configure_view(
    stroke=None
)

final_chart


Agora faremos outra visualização, dessa vez visando analisar os pesos e alturas de cada atleta.

Primeiro faremos novamente as análises estatísticas para as váriaveis de peso.


In [ ]:
dados['peso'].describe()

Temos algumas estatísticas mas seriam melhores se fossem separadas por esporte já que existem esportes que naturalmente favorecem pessoas mais pesadas ou mais leves.

In [ ]:
dados.groupby('esporte')['peso'].describe()

Percebeos que assim como na altura o Alpinismo e Patrulha Militar não tem dados e o motivo é o mesmo citado anteriormente.

Ademais, percebe-se que os esportes que tiveram maior média de alturas tendem a ter maiores pesos também, o que é completamente válido já que pessoas maiores tendem a pesar mais em média. Contudo, aida são precisas análises mais detalhadas para observar a real distribuição de pesos e se há alguma anormalidade inesperada.

Faremos agora uma análise, similar com a feita com as alturas, que abordará mais fortemente o quesito de medalhas ganhas pelos atletas.

In [ ]:
dados.groupby(['esporte', 'medalha'])['peso'].describe()

Após ver esses dados é claro observar que dado um determinado esporte as diferenças médias de pesos são estatisticamente insgnificantes visto o desvio padrão.

Para conseguirmos enfim ter certeza sobre co o é essa distribuição de pesos dos atletas faremos uma visualização.

In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso'),
        alt.Tooltip('medalha')
        ]
).properties(
    title= 'Peso dos altetas'
)

Aqui já podemos perceber que a distribuição se aproxima da de altura em certo sentido mas por outro lado parece ter uma concentração entorno da média mas um desvio padrão maior como as estatísticas indicavam.

In [ ]:
alt.Chart(dados).mark_circle().transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso'),
        alt.Tooltip('medalha')
        ],
    color=alt.Color('medalha')
).properties(
    title= 'Peso dos altetas medalhistas'
)

Os atletas medalhistas aparentam ter a mesma distribuição.'

In [ ]:
alt.Chart(dados).mark_circle().encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso')
        ]
).facet(
    facet='esporte:N',
    columns=3
)

Com a separação por esportes já consegiumos ver uma diferenciação mais acentuada que na altura cmoo a diferença de patinação no gelo e bobsleigh

In [ ]:
alt.Chart(dados).mark_bar().encode(
    alt.X('peso',
          scale=alt.Scale(domain=[30, 150]), title='Peso (kg)'),
    alt.Y('count()', title= 'Número de ocorreências'),
    tooltip=[
        alt.Tooltip('nome_atleta'),
        alt.Tooltip('esporte'),
        alt.Tooltip('pais'),
        alt.Tooltip('peso')
        ]
)

Com essa visualização podemos perceber uma espécie de distribuição normal com mais "outliers" para o lado direito e concentração maior na parte esquerda.

In [ ]:
chart2 = alt.Chart(dados).mark_point(filled=True).transform_filter(
    alt.datum.medalha != None
).encode(
    alt.X('altura', scale=alt.Scale(domain=[140, 205]), title='Altura (cm)'),
    alt.Y('peso', scale=alt.Scale(domain=[30, 130]), title='Peso (kg)'),
    alt.Column('esporte', title='Esporte'),
    color=alt.Color('medalha', title='Medalha'
    ),
    tooltip=[
        alt.Tooltip('nome_atleta', title='Atleta'),
        alt.Tooltip('medalha', title='Medalha'),
    ]
).properties(
    title='Altura x Peso dos atletas medalhistas (usando window ranking)'
)

chart2


Essa visualização nos permite observar um comportamento próximo do linear associando a altura com o peso e como esse comportamento linear se dá em cada esporte diferente, mas não nos permite ter boa visualização de alguma possíve distribuição dos medalhistas

In [ ]:
chart_fixed = alt.Chart(dados).mark_point(filled=True).transform_filter(
    alt.datum.medalha != None
).transform_calculate(
    esporte_pt="""{'Alpine Skiing': 'Esqui Alpino',
        'Biathlon': 'Biatlo',
        'Bobsleigh': 'Bobsleigh',
        'Cross Country Skiing': 'Esqui Cross Country',
        'Curling': 'Curling',
        'Figure Skating': 'Patinação Artística',
        'Freestyle Skiing': 'Esqui Estilo Livre',
        'Ice Hockey': 'Hóquei no gelo',
        'Luge': 'Luge',
        'Nordic Combined': 'Combinado Nórdico',
        'Short Track Speed Skating': 'Patinação de velocidade curta',
        'Skeleton': 'Skeleton',
        'Ski Jumping': 'Salto de Esqui',
        'Snowboarding': 'Snowboarding',
        'Speed Skating': 'Patinação de velocidade'
    }[datum.esporte]"""
).transform_calculate(
    medalha_valor="if(datum.medalha == 'Gold', 3, if(datum.medalha == 'Silver', 2, if(datum.medalha == 'Bronze', 1, 0)))"
).transform_joinaggregate(
    max_medalha_valor="max(medalha_valor)",
    groupby=["nome_atleta"]
).transform_filter(
    alt.datum.medalha_valor == alt.datum.max_medalha_valor
).encode(
    alt.X('altura', scale=alt.Scale(domain=[140, 205]), title='Altura (cm)'),
    alt.Y('peso', scale=alt.Scale(domain=[30, 130]), title='Peso (kg)'),
    alt.Column('esporte_pt:N', title='Esporte'),
    color=alt.Color('medalha:N', title='Medalha', scale=alt.Scale(
        domain=['Gold', 'Silver', 'Bronze'],
        range=['#FFD700', '#C0C0C0', '#CD7F32']
    ), legend=alt.Legend(
        titleFontSize=14,
        labelFontSize=12,
        symbolSize=200,
        orient='left',
        labelExpr="{'Gold': 'Ouro', 'Silver': 'Prata', 'Bronze': 'Bronze'}[datum.label]"
    )),
    tooltip=[
        alt.Tooltip('nome_atleta', title='Atleta'),
        alt.Tooltip('medalha', title='Medalha'),
        alt.Tooltip('medalha_valor:Q', title='Medalha Valor'),
        alt.Tooltip('max_medalha_valor:Q', title='Max Medalha Valor'),
        alt.Tooltip('esporte', title='esporte')
    ]
).properties(
    title='Altura x Peso dos atletas medalhistas (melhor medalha por atleta)'
)

chart_fixed


Agora podemos ter melhor noção de como se distribuem os medalhistas de ouro prata e bronze em cada esporte e podemos perceber que, por exemplo no esporte hóquei no gelo medalhistas de ouro tendem a ter uma estatura e peso maior que é um esporte extremamente físico mas já no curling percebemos um espaçamento maior já que é um esporte que exige menos do condicionamento físico.

Por fim pudemos perceber que os esportes têm grande influência sobre as medidas físicas de seus atletas, espcialmente de seus medalhista, inclusive quais tipos de medalhistas.

## Lucas Paulo Gonçalves

Farei dois gráficos usando o dataset cars, do vega_datasets.

Começo importando as bibliotecas necessárias e selecionando o dataset:

In [ ]:
import altair as alt
import pandas as pd
import vega_datasets as vdts

cars = vdts.data.cars()

In [ ]:
cars.head()

Para este primeiro gráfico, quero ver relações entre volume do motor (Displacement) e aceleração (Acceleration).

Estas são as estatísticas de resumo dos dois campos:

In [ ]:
print(cars[['Displacement', 'Acceleration']].describe(), '\n', cars['Origin'].mode())

Começo fazendo um plot simples dos dados:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration')
    )

Para este gráfico, também quero ver as origens de cada carro. Como neste dataset tenho apenas 3 origens possíveis (USA, Europe, Japan), usarei uma seleção de cores para diferenciar cada um:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration'),
    color = alt.Color('Origin:N')
    )

Para complementar, altero as cores para cores que julgo mais adequadas para esta divisão:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration'),
    color = alt.Color('Origin:N',
                       scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
    )
    )

Ter a legenda à direita não me agrada, então a movo para cima:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Displacement:Q'),
    alt.Y('Acceleration'),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(orient='top'))
    )

Como todos os nomes estão em inglês, adicionarei títulos ao gráfico, à legenda e a cada eixo:

In [ ]:
alt.Chart(cars, title = 'Aceleração do motor por volume').mark_circle().encode(
    alt.X('Displacement:Q', title = 'Volume do motor (pol³)'),
    alt.Y('Acceleration:Q', title = 'Aceleração do motor (mph/s)'),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
    )

Normalmente, quão menor um motor é em volume, melhor ele performa, e quão maior a aceleração de um carro, melhor. Seguindo a ideia de que o melhor fica para cima/à direita, inverto a ordem do eixo X para que os motores menores em volume fiquem mais à direita:

In [ ]:
alt.Chart(cars, title = 'Aceleração do motor por volume').mark_circle().encode(
    alt.X('Displacement:Q', title = 'Volume do motor (pol³)', sort = 'descending'),
    alt.Y('Acceleration:Q', title = 'Aceleração do motor (mph/s)'),
    color = alt.Color('Origin:N',
                       scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
    )

Para finalizar, elimino os espaços brancos  para que possa melhor visualizar meus dados no gráfico. Para isso, tiro os intervalos que não contém pontos definindo o domínio de cada eixo:

In [ ]:
alt.Chart(cars, title = 'Aceleração do motor por volume').mark_circle().encode(
    alt.X('Displacement:Q', title = 'Volume do motor (pol³)', sort = 'descending', scale = alt.Scale(domain = [60, 470])),
    alt.Y('Acceleration:Q', title = 'Aceleração do motor (mph/s)', scale = alt.Scale(domain = [7, 26])),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
    )

Neste gráfico, é possível ver que os pontos são bem mais densos na região do volume entre 200 e 100 pol³. Faço uma análise unidimensional por meio de um histograma para confirmar este fato:

In [ ]:
alt.Chart(cars, title = 'Quantidade de carros por volume do motor').mark_bar().encode(
    alt.X('Displacement', title = 'Volume do motor (pol³)', sort = 'descending', bin = True),
    alt.Y('count()', title = 'Frequência')
)

Um fato que podemos obter visualmente no gráfico é que, em média, quão menor em volume o motor é, maior a aceleração. Podemos associar essa variação em aceleração ao peso do carro - quão mais leve o carro, maior deveria ser sua aceleração. Queremos, então associar o volume do motor ao peso do carro - quão maior o volume, maior o peso. Posso fazer isso numa análise bidimensional das duas variáveis, peso (Weight_in_lbs) e volume do motor (Displacement).

Para isso, farei um gráfico simples de pontos do peso pelo volume do motor:

In [ ]:
alt.Chart(cars, title = 'Peso dos carros por volume do motor').mark_circle().encode(
    alt.X('Displacement', title = 'Volume do motor (pol³)', sort = 'descending'),
    alt.Y('Weight_in_lbs', title = 'Peso (lbs)')
)

 Neste gráfico, já conseguimos ver uma relação clara entre volume do motor e peso. Para confirmar, faço a matriz de correlação desses dois campos de dados:

In [ ]:
cars[['Displacement', 'Weight_in_lbs']].corr()

O coeficiente de correlação do peso com o volume do motor é de 0.932, o que mostra uma relação linear entre peso do carro e volume do motor.

Gostaria de ver agora, no segundo gráfico, uma relação entre data de produção e peso.

Estas são as estatísticas de resumo:

In [ ]:
cars[['Year', 'Weight_in_lbs']].describe()

Começo fazendo um plot básico:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Year:T'),
    alt.Y('Weight_in_lbs')
)

Adiciono uma seleção de cor baseada na origem do carro, do mesmo esquema do gráfico anterior:

In [ ]:
alt.Chart(cars).mark_circle().encode(
    alt.X('Year:T'),
    alt.Y('Weight_in_lbs'),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
)

Adiciono títulos aos eixos e ao gráfico e limito o domínio do peso, já que este tem mínimo em 1613:

In [ ]:
alt.Chart(cars, title = 'Peso do carro por ano').mark_circle().encode(
    alt.X('Year:T', title = 'Ano de produção'),
    alt.Y('Weight_in_lbs', title = 'Peso (lbs)', scale = alt.Scale(domain = [1600, 5200])),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
)

Este gráfico já está útil, porém quero visualizar melhor os pontos na parte inferior, logo vou aplicar uma escala logarítmica de base 10 no eixo Y:

In [ ]:
alt.Chart(cars, title = 'Peso do carro por ano').mark_circle().encode(
    alt.X('Year:T', title = 'Ano de produção'),
    alt.Y('Weight_in_lbs', title = 'Peso (lbs)', scale = alt.Scale(domain = [1600, 5200], type = 'log')),
    color = alt.Color('Origin:N',
                      scale = alt.Scale(range = ['rgb(255, 221, 0)', 'rgb(255, 0, 0)', 'rgb(16, 3, 153)']),
                      legend = alt.Legend(title = 'Origem do Carro', orient='top'))
)

Com este gráfico, podemos ver e afirmar que os carros americanos começaram consistentemente mais pesados que os carros europeus e japoneses, porem, com o passar do tempo, tem alcançado medidas similares. Isso explica o comportamento visto no primeiro gráfico - carros com maior volume de motor (mais pesados) são predominantemente americanos, porém são definitivamente mais antigos. Faço uma análise unidimensional dos carros americanos para confirmar que existe uma quantidade considerável de carros americanos em todas as classes de peso:

In [ ]:
alt.Chart(cars[cars['Origin'] == 'USA'], title = 'Histograma de frequência de carros americanos por classe de peso').mark_bar().encode(
    alt.X('Weight_in_lbs', title = 'Peso (lbs)', bin = True),
    alt.Y('count()', title = 'Frequência')
)

Podemos ver que carros americanos tem um peso bem distribuído, o que pode justificar a grande dispersão dos pontos americanos no gráfico 1. Para finalizar a lógica, farei uma análise bidimensional de carros americanos baseado na kilometragem do carro (`Horsepower`) e o volume do motor:

In [ ]:
alt.Chart(cars[cars['Origin'] == 'USA'], title = 'Potência em cavalos por volume do motor').mark_circle().encode(
    alt.X('Displacement', title = 'Volume do motor (m³)'),
    alt.Y('Miles_per_Gallon', title = 'Kilometragem (mpg)')
)

A partir desses dois gráficos gerados, podemos chegar a uma conjectura: Carros americanos vem evoluindo com o tempo. Eram originalmente bem mais pesados em comparação aos europeus e japoneses, mas vem consistentemente se tornando mais leves. Essa diminuição de peso é um grande sinal de aumento de qualidade, já que carros mais leves tem motores menores, que por sua vez aceleram mais e bebem menos, o que quer dizer menor consumo de combustível e logo menos poluição! Logo, a partir apenas desses dados, podemos dizer que carros americanos vem se tornando menos poluentes com o passar dos anos!